In [1005]:
import requests
import re
import sys
import time
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import subprocess
import time
import ipaddress
from datetime import datetime
import shlex
import math
from typing import Optional, Dict, Any
import argparse, secrets, hashlib

In [1006]:
# Floodlight Controller Details
FLOODLIGHT_CONTROLLER_IP = "127.0.0.1"
FLOODLIGHT_CONTROLLER_PORT = 8080
FLOODLIGHT_BASE_URL = f"http://{FLOODLIGHT_CONTROLLER_IP}:{FLOODLIGHT_CONTROLLER_PORT}"

# API Paths
STATIC_FLOW_PUSHER_URL = f"{FLOODLIGHT_BASE_URL}/wm/staticflowpusher"
STATIC_FLOW_LIST_URL = f"{STATIC_FLOW_PUSHER_URL}/list"
STATIC_FLOW_CLEAR_URL = f"{STATIC_FLOW_PUSHER_URL}/clear"
SWITCH_STATS_FLOW_URL = f"{FLOODLIGHT_BASE_URL}/wm/core/switch"
SWITCHES_URL = f"{FLOODLIGHT_BASE_URL}/wm/core/controller/switches/json"

# Define sudo password
sudo_password = "test@irciss008" #your localhost password

In [1007]:
ip_to_host = {
    "10.0.1.1": "h1",
    "10.0.1.2": "h2",
    "10.0.1.3": "h3",
    "10.0.1.4": "h4"
            }

host_to_ip = {
    "h1": "10.0.1.1",
    "h2": "10.0.1.2",
    "h3": "10.0.1.3",
    "h4": "10.0.1.4",
}

# --- Diamond topology wiring helpers (Floodlight format) ---

SW_OF = {
    "1": "00:00:00:00:00:00:00:01",  # s1
    "2": "00:00:00:00:00:00:00:02",  # s2
    "3": "00:00:00:00:00:00:00:03",  # s3
    "4": "00:00:00:00:00:00:00:04",  # s4
}

HOSTS = {
    "h1": "10.0.1.1",
    "h2": "10.0.1.2",
    "h3": "10.0.1.3",
    "h4": "10.0.1.4",
}

# Host attachment (edge switch, access port) from your Mininet build
# This now uses the correct DPID format as keys
HOST_ATTACH = {
    HOSTS["h1"]: (SW_OF["1"], 3),  # h1 -> s1:3
    HOSTS["h2"]: (SW_OF["1"], 4),  # h2 -> s1:4
    HOSTS["h3"]: (SW_OF["4"], 3),  # h3 -> s4:3
    HOSTS["h4"]: (SW_OF["4"], 4),  # h4 -> s4:4
}

switch_id_for_llm_assurance = None
llm_caller_flag = 0
gl_flow_name = "" # Use flow name for tracking, not cookie

In [1008]:
def dump_all_operational_flows(dpid_str: str) -> str:
    """Helper to get a formatted string of all operational flows on a switch."""
    try:
        url = f"{SWITCH_STATS_FLOW_URL}/{dpid_str}/flow/json"
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        data = r.json()
        # Pretty-print the JSON for readability
        return json.dumps(data.get("flows", []), indent=2)
    except Exception as e:
        return f"Error dumping flows for {dpid_str}: {e}"

def fl_get(url):
    """Generic GET request for Floodlight."""
    try:
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        return r.json()
    except requests.RequestException as e:
        print(f"Error in GET {url}: {e}")
        return None

def fl_post(url, json_payload):
    """Generic POST request for Floodlight."""
    try:
        r = requests.post(url, json=json_payload, timeout=5)
        r.raise_for_status()
        return r.json()
    except requests.RequestException as e:
        print(f"Error in POST {url}: {e} - Payload: {json_payload}")
        return None

def add_flow_floodlight(flow):
    """
    Pushes a single static flow rule to Floodlight.
    'flow' must be a valid Floodlight static flow JSON object.
    """
    response = fl_post(f"{STATIC_FLOW_PUSHER_URL}/json", flow)
    if response and response.get("status") == "Entry pushed":
        return True
    print(f"Warning: Add flow failed: {response}")
    return False

def delete_flow_floodlight_alternative(flow_name):
    """
    Deletes a static flow rule from Floodlight by its 'name'.
    """
    payload = {"remove": {"name": flow_name}}
    response = fl_post(f"{STATIC_FLOW_PUSHER_URL}/json", payload)
    if response and "Entry removed" in response.get("status", ""):
        return True
    # It might also be a DELETE request, but POST with "remove" is common
    # Alternative:
    # r = requests.delete(f"{STATIC_FLOW_PUSHER_URL}/json", json={"name": flow_name})
    # if r.status_code == 200: return True
    
    print(f"Warning: Delete flow failed for name {flow_name}: {response}")
    return False

def delete_flow_floodlight(flow_name):
    url = f"{STATIC_FLOW_PUSHER_URL}/json"
    try:
        r = requests.delete(url, json={"name": flow_name}, timeout=5)
        if r.ok: return True
        # fallback(s)
        r = requests.post(url, json={"name": flow_name, "command": "delete"}, timeout=5)
        if r.ok: return True
        r = requests.post(url, json={"name": flow_name, "action": "remove"}, timeout=5)
        return r.ok
    except requests.RequestException as e:
        print(f"Error deleting {flow_name}: {e}")
        return False


def get_static_flows_floodlight(dpid_str):
    """
    Gets all *static* flows for a given switch DPID.
    """
    data = fl_get(f"{STATIC_FLOW_LIST_URL}/{dpid_str}/json")
    return data.get(dpid_str, []) if data else []

def get_operational_flows_floodlight(dpid_str):
    """
    Gets all *operational* flows (with stats) from a switch.
    """
    data = fl_get(f"{SWITCH_STATS_FLOW_URL}/{dpid_str}/flow/json")
    return data.get("flows", []) if data else []

def wait_for_dpids(expect_dpids, timeout=30):
    """
    Waits for a set of DPIDs (colon-hex) to connect to Floodlight.
    """
    want = set(expect_dpids)
    start = time.time()
    seen_dpids = set()
    while time.time() - start < timeout:
        switches_data = fl_get(SWITCHES_URL)
        if switches_data:
            seen_dpids = {s.get("dpid") for s in switches_data}
            if want.issubset(seen_dpids):
                return sorted(seen_dpids)
        time.sleep(1)
    return sorted(seen_dpids)

def find_flow_by_name(flow_dump, name_to_find):
    """Finds a flow in a list of static flows by its 'name'."""
    for flow_entry_dict in flow_dump:
        if name_to_find in flow_entry_dict:
            return flow_entry_dict[name_to_find]
    return None

def exists_floodlight(dpid_str, flow_name, retries=3, delay=0.5):
    """
    Checks if a static flow exists on a switch by its 'name'.
    Returns (True, found_flow) or (False, None).
    """
    for _ in range(retries):
        dump = get_static_flows_floodlight(dpid_str)
        found = find_flow_by_name(dump, flow_name)
        if found:
            return True, found
        time.sleep(delay)
    return False, None

def fingerprint(flow):
    """
    (This helper is fine, but we prefer using 'name'.)
    Generates a hash for a flow rule.
    """
    # We must adapt this to the flat Floodlight JSON structure
    # Let's fingerprint the *match* and *actions*
    
    # Extract match keys (anything not in the control set)
    control_keys = {"switch", "name", "active", "priority", "actions"}
    match_part = {k: v for k, v in flow.items() if k not in control_keys}
    
    h = hashlib.sha1(json.dumps({
        "priority": flow.get("priority"),
        "match": match_part,
        "actions": flow.get("actions", "")
    }, sort_keys=True).encode()).hexdigest()
    return h[:12]

In [1009]:
TRANSLATION_PROMPT_FLOODLIGHT = """
You are a meticulous network engineer. Convert the user's intent into a single Floodlight (Static Entry Pusher, OpenFlow 1.0) flow rule as a JSON object ONLY (no code fences, no comments, no extra text).

Return exactly ONE JSON object with ONLY these keys.

Required keys (all strings):
- "switch": "00:00:00:00:00:00:00:0X"   (DPID; e.g., N → "...:0N")
- "name": "flow-<short-descriptor>"
- "active": "true"
- "priority": "<0-65535>"

Allowed match keys (strings; include ONLY those needed):
- "in_port"
- "eth_type"            (IPv4=0x0800, IPv6=0x86dd, ARP=0x0806)
- "eth_src", "eth_dst"
- "ip_proto"            (TCP=6, UDP=17, ICMP=1)
- "ipv4_src", "ipv4_dst", "ipv6_src", "ipv6_dst"
- "tcp_src", "tcp_dst", "udp_src", "udp_dst", "sctp_src", "sctp_dst"
- "icmpv4_type", "icmpv4_code"
- "arp_opcode", "arp_spa", "arp_tpa", "arp_sha", "arp_tha"
- "eth_vlan_vid", "eth_vlan_pcp"

Allowed action key:
- "actions": "<comma-separated actions>" (omit this key entirely to DROP)

Allowed action values (comma-separated in a single string):
- "output=X"
- "set_queue=Y"
- "push_vlan=0x8100"
- "pop_vlan"
- "set_vlan_vid=VID"

Interpretation rules:
- “in/on switch N” → "switch":"00:00:00:00:00:00:00:0N".
- If you use any IP/L4 fields and "eth_type" is unspecified, set "eth_type":"0x0800".
- Protocol cues: TCP→"ip_proto":"6", UDP→"17", ICMP→"1".
- Services: HTTP→"tcp_dst":"80", HTTPS→"443", SSH→"22", DNS→"udp_dst":"53", DHCP→"udp_dst":"67" (server) or "68" (client), ping→"icmpv4_type":"8".
- “forward/send out/through port X” → "actions":"output=X".
- QoS “queue Y” → "actions":"set_queue=Y,output=X".
- “block/deny/drop” → OMIT the "actions" key.
- Use ONLY fields implied by the intent (plus the defaults above). All keys and values are strings.

Priority guidance (pick the most appropriate):
- "300" → explicit block/deny
- "200" → highly specific forwarding with QoS
- "100" → specific L3/L4 matches
- "50"  → less-specific catch-alls

Constraints:
- Output must be a single valid JSON object, nothing else.
- Do not invent fields, do not include comments or markdown.
"""


In [1010]:
CONFLICT_PROMPT_FLOODLIGHT = """
You are an expert network engineer analyzing two Floodlight (OpenFlow) flow rules to decide whether they conflict.

PRIMARY DECISION
A conflict exists if BOTH are true:
1) The rules apply to the same switch (equal "switch"; if both specify "table", tables must be equal too).
2) Their match conditions overlap (there exists at least one packet that matches both),
   AND either:
   a) their actions differ, or
   b) they are redundant (same action and one rule’s match is a subset of the other).

DEFINITIONS & NORMALIZATION
- Floodlight drop = the "actions" key is absent. (Empty string is treated as drop ONLY if present in the input.)
- Actions are a comma-separated list in a single string (e.g., "set_queue=1,output=2"). Compare actions as an unordered set of tokens; ignore whitespace and token order.
- Consider only top-level match keys (e.g., "in_port", "eth_type", "ip_proto", "ipv4_src", "ipv4_dst", "tcp_dst", "udp_dst", "eth_vlan_vid", etc.). Keys like "name", "active", "priority" do not affect matching.
- Normalize values before comparison:
  • eth_type: "0x800" == "0x0800" (IPv4).  
  • ip_proto: accept "6"/"tcp", "17"/"udp", "1"/"icmp".  
  • ports and priorities: compare numerically even if strings.  
  • IPs may be host (/32) or CIDR; treat "10.0.0.1" as "10.0.0.1/32".
- Match overlap rules:
  • If "eth_type" differ (e.g., IPv4 vs IPv6), no overlap.  
  • If "ip_proto" differ (e.g., TCP vs UDP), no overlap.  
  • If both specify "in_port" with different values, no overlap.  
  • IP prefixes overlap if their CIDRs intersect (e.g., 10.0.0.0/24 overlaps 10.0.0.1/32).  
  • TCP/UDP port equality is required when both specify the same L4 field.  
  • VLAN: "eth_vlan_vid" must be equal if both specify it; otherwise the more general one (no VLAN constraint) overlaps the specific one.  
  • Absence of a field makes a rule more general on that dimension.

GUIDANCE
- Different "switch" → no conflict. If both specify "table" and they differ → no conflict.  
- Overlap + different actions → conflict.  
- Overlap + same actions:
    • If one match ⊂ the other → conflict (Redundancy).  
    • If partial overlap without subset and same actions → treat as no conflict for this binary decision.
- Priority does not affect the YES/NO decision, but may be used in the explanation (e.g., “more general rule may shadow a specific rule if higher priority”).

EXAMPLES

Example A: Conflict (Different Action)
F1: {"switch":"...:01","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=1"}
F2: {"switch":"...:01","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=2"}
Output: {"conflict_status":1,"conflict_explanation":"Same switch and identical match; actions differ (output=1 vs output=2)."}

Example B: No Conflict (Different Switch)
F1: {"switch":"...:01","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=3"}
F2: {"switch":"...:02","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=3"}
Output: {"conflict_status":0,"conflict_explanation":"Different switches."}

Example C: Conflict (Redundancy)
F1: {"switch":"...:04","eth_type":"0x0800","ip_proto":"6","tcp_dst":"80","actions":"output=2"}
F2: {"switch":"...:04","eth_type":"0x0800","ip_proto":"6","actions":"output=2"}
Output: {"conflict_status":1,"conflict_explanation":"Specific rule is subset of a more general rule with the same action (redundancy)."}

Example D: No Conflict (Protocol Mismatch)
F1: {"switch":"...:01","eth_type":"0x0800","ip_proto":"6","tcp_dst":"22","actions":"output=1"}
F2: {"switch":"...:01","eth_type":"0x0800","ip_proto":"17","udp_dst":"53","actions":"output=1"}
Output: {"conflict_status":0,"conflict_explanation":"TCP vs UDP are mutually exclusive."}

INPUT FORMAT
You will be provided with two JSON flow rules:

Flow 1:
<JSON>

Flow 2:
<JSON>

EXPECTED OUTPUT (strict JSON only):
{
  "conflict_status": <0 or 1>,
  "conflict_explanation": "<brief reason or empty string>"
}
NO EXTRA TEXT.
"""


In [1011]:
SLICING_PROMPT = """You are tasked with analyzing a natural language intent to determine if it contains a command to create or use a queue/slice in an OpenFlow switch. You should respond in JSON format.

### Rules for Interpretation:
1. **Queue/Slice Detection:**  
   - The intent is considered related to queue/slice if it contains commands such as:
     - "create queue", "create slices", "slice the network", "implement slicing", "slice the flow", "make flowspace slicing", "do slicing", "slice", "implement queue", "do queuing", "assign queue", "assign slice", or any similar phrasing.
   - If the intent does not mention creating or using a queue/slice, set the field `"use_queue"` to `0`.

2. **Switch, Queue, and Port Identification (Data-plane port/interface):**  
   - If the intent specifies a **switch ID** (e.g., "switch 4" or "openflow:4" or "node 4" or "openflow 4"), populate the `"switch_id"` field with its value.  
   - If the intent specifies a **Queue ID or slice ID** (e.g., "queue 4" or "4th queue" or "fourth queue" or "slice 1" or "first slice"), populate the `"queue_id"` field with its value.  
   - If the intent specifies a **port ID** referring to the device interface/output (e.g., "port 2" or "interface 2" or "ethernet 2" or "output node connector 2" or "second port" or "second interface"), populate the `"port_id"` field with its value. If there are multiple instances of "port_id" present, take the one which indicates the **output port or outgoing interface**.
   - If the intent does not specify a switch ID or queue ID or port ID, set the respective field to an empty string (`""`).

3. **Traffic Type and L4 Destination Port Extraction (Protocol/Service):**
   - Detect the **Layer-4 protocol** if mentioned: `"tcp"` or `"udp"`. Populate this in `"traffic_type"`. If not specified, set `"traffic_type"` to `""`.
   - Detect the **destination application port number** (Layer-4 port), if specified as a number (e.g., "port 80", "UDP port 53") or implied via a service reference such as "HTTP (TCP port 80)". Populate this number (as a string) in `"l4_port"`.  
   - Do **not** confuse the L4 port (e.g., 80/53) with the device/interface port (e.g., switch port 2). The former goes to `"l4_port"`, the latter to `"port_id"`.
   - If multiple L4 ports are mentioned, prefer the **destination/service port** used by the traffic selector (e.g., "traffic destined for port 80"). If still ambiguous, choose the first explicit destination/service port mentioned.
   - If the L4 destination port is not specified, set `"l4_port"` to `""`.

4. **Negative Constraint**: Intents that only contain commands like "block", "drop", "deny", or "forward" without any explicit mention of "queue" or "slice" are not queue-related, and use_queue must be 0.

5. **Output Format:**  
   - Respond strictly in valid JSON format adhering to the following schema:

```json
{
  "use_queue": <integer>,
  "switch_id": "<string>",
  "queue_id": "<string>",
  "port_id": "<string>",
  "traffic_type": "<string>",
  "l4_port": "<string>"
}

Field Description:
use_queue: 1 if the intent commands to create or use a queue/slice, 0 otherwise.
switch_id: Switch ID if specified in the intent, otherwise "".
queue_id: Queue/Slice ID if specified in the intent, otherwise "".
port_id: Device/interface port if specified (output/outgoing preferred), otherwise "".
traffic_type: "tcp" or "udp" if specified, otherwise "".
l4_port: Destination application port number (e.g., "80", "53") if specified, otherwise "".

No Additional Text:
Do not include any comments, explanations, or outputs outside the JSON format.

Example Inputs and Outputs:

Input Intent:
"Create a queue in switch 4 on port 3 for slicing the flow."
Output:
{
"use_queue": 1,
"switch_id": "switch 4",
"queue_id": "",
"port_id": "port 3",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Send all video traffic through queue 0 of openflow:2."
Output:
{
"use_queue": 1,
"switch_id": "openflow 2",
"queue_id": "0",
"port_id": "",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Configure switch 5 for traffic management."
Output:
{
"use_queue": 0,
"switch_id": "switch 5",
"queue_id": "",
"port_id": "",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Monitor traffic flow on port 1."
Output:
{
"use_queue": 0,
"switch_id": "",
"queue_id": "",
"port_id": "port 1",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
Output:
{
"use_queue": 1,
"switch_id": "switch 3",
"queue_id": "0",
"port_id": "interface 2",
"traffic_type": "tcp",
"l4_port": "80"
}

Input Intent:
"For node 1, route HTTP (TCP port 80) traffic targeting 10.0.0.3/32 through port 2 with traffic assigned to queue 0."
Output:
{
"use_queue": 1,
"switch_id": "node 1",
"queue_id": "0",
"port_id": "port 2",
"traffic_type": "tcp",
"l4_port": "80"
}

Input Intent:
"Forward UDP traffic on port 53 destined for 10.0.0.9 via interface 5 of switch 2, assigning it to queue 3."
Output:
{
"use_queue": 1,
"switch_id": "switch 2",
"queue_id": "3",
"port_id": "interface 5",
"traffic_type": "udp",
"l4_port": "53"
}
"""

In [1012]:
my_models_translate_real = [
"codestral:22b",
"command-r:35b",
"huihui_ai/qwq-abliterated:latest",
]

my_models_conflict_real = [  
"huihui_ai/qwq-fusion:latest",
"qwq:latest"
]

context_examples = [3, 6]

default_model = "llama2:7b"

ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [1013]:
# Load custom dataset from CSV
custom_dataset = pd.read_csv('Intent2Flow-Floodlight.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
#trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)
trainset = custom_dataset

In [1014]:
def append_intent_to_store(
    file_path,
    nl_intent,
    json_flow_rule,
    device_id,
    flow_id,
    intent_type,
    intent_specificity
):
    """
    Appends an intent record to the IntentStore file in JSONL format.
    """
    record = {
        "nl_intent": nl_intent,
        "json_flow_rule": json_flow_rule,
        "device_id": device_id,
        "flow_id": flow_id,
        "intent_type": intent_type,
        "intent_specificity": intent_specificity
    }
    with open(file_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record) + "\n")

def read_intents_from_store(file_path):
    """
    Yields each intent record from the IntentStore file.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            intent = json.loads(line)
            yield intent

In [1015]:
def run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json):
    system_prompt = CONFLICT_PROMPT_FLOODLIGHT

    for model in my_models_conflict_real:
        count = 0
        while True:
            count+=1
            try:
                time.sleep(0.1)

                response = client.generate(model=model,
                    options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                    stream=False,
                    system=system_prompt,
                    prompt=f"Flow 1:\n{json.dumps(existing_intent_flow_json, indent=2)}\n\nFlow 2:\n{json.dumps(new_intent_flow_json, indent=2)}",
                    format='json'
                )

                output = response['response'].strip()

                response_json = json.loads(output)

                if 'conflict_status' not in response_json:
                    #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                    break
                else:
                    valid_conflict_response = True
                    conflict_status = response_json.get('conflict_status', 0)
                    # Ensure conflict_status is an integer
                    if isinstance(conflict_status, str):
                        conflict_status = int(conflict_status)

                    return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

            except Exception as e:
                print("Exception found: ", e)
                sys.stdout.flush()
                if(count<15):
                    continue
                else:
                    print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                    break               
    
    return False, None, None

In [1016]:
def conflict(device_dpid_str, new_intent_flow_json):
    """
    Checks for conflicts against operational flows.
    device_dpid_str must be in colon-hex format.
    """

    # Use your new Floodlight helper function
    #existing_flows = get_operational_flows_floodlight(device_dpid_str) # <--- NEW

    # The rest of this function must be adapted for the operational flow format
    # Floodlight's operational flow format is different from its static flow format.
    # It has 'match' and 'actions' as sub-objects.
    # The CONFLICT_PROMPT_FLOODLIGHT is designed for the STATIC format.
    
    # This is a problem. The conflict LLM is trained on the *static* format,
    # but the 'existing_flows' are in *operational* format.
    
    # This is solved as the static flow rules can be fetched and used instaed of operational ones.
    
    # Let's try fetching STATIC flows instead for a true apples-to-apples comparison.
    existing_static_flows = get_static_flows_floodlight(device_dpid_str)
    
    #
    # *** DEBUGGING NOTE: ***
    # If you want to conflict against ALL flows (including those from
    # reactive apps), use get_operational_flows_floodlight() and update
    # the conflict prompt to handle the { "match": {...}, "actions": [...] } format.
    # For now, we conflict against other static flows.
    #
    
    for existing_flow in existing_static_flows:

        # --- START: ROBUST HARD-CODED EXCLUSION ---
        # This filter needs to check the flat JSON structure
        try:
            # new_intent_flow_json is flat
            match1 = new_intent_flow_json
            # existing_flow is also flat
            match2 = existing_flow

            dl_dst_1 = str(match1.get('eth_dst', ''))
            dl_dst_2 = str(match2.get('eth_dst', ''))
            
            is_m1_l2_special = dl_dst_1.startswith('01:80:c2:00:00:0') 
            is_m2_l2_special = dl_dst_2.startswith('01:80:c2:00:00:0')

            eth_type_1 = str(match1.get('eth_type', '')).lower()
            eth_type_2 = str(match2.get('eth_type', '')).lower()
            
            is_m1_ip_arp = eth_type_1 in ['2048', '0x0800', '2054', '0x0806'] or 'ipv4_dst' in match1 or 'ipv4_src' in match1
            is_m2_ip_arp = eth_type_2 in ['2048', '0x0800', '2054', '0x0806'] or 'ipv4_dst' in match2 or 'ipv4_src' in match2

            if (is_m1_l2_special and is_m2_ip_arp) or (is_m1_ip_arp and is_m2_l2_special):
                continue
        
        except Exception as e:
            print(f"Warning: Pre-filter check failed: {e}")
            pass
        # --- END: ROBUST HARD-CODED EXCLUSION ---

        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(existing_flow, new_intent_flow_json)

        if (valid_conflict_response == False):
            return 2, None, None
        elif (conflict_status == 1):
            return conflict_status, conflict_details, existing_flow

    return 0, None, None

In [1017]:
def run_LLM_IBN(intent, device_id_str):
    
    # NOTE: 'device_id_str' is the DPID, but the LLM prompt
    # is designed to extract this from the intent text itself.
    # We pass it here in case we want to use it later.

    for num_examples in context_examples:
        for model in my_models_translate_real:
            
            # !! REMINDER !!
            # The 'trainset' (from Intent2Flow-Floodlight.csv) MUST
            # have examples that match the TRANSLATION_PROMPT_FLOODLIGHT format.
            example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
                [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
                ollama_emb,
                Chroma,
                input_keys=["instruction"],
                k=num_examples,
                vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
                )
            # Clear and add all remaining examples from the trainset
            example_selector.vectorstore.reset_collection()
            
            for _, row in trainset.iterrows():
                example_selector.add_example({
                    "instruction": row["instruction"],
                    "output": row["output"]
                })
            
            # Use the new Floodlight-compatible prompt
            system_prompt = TRANSLATION_PROMPT_FLOODLIGHT
            count = 0

            while True:
                count+=1
                try:
                    time.sleep(0.1)
                    example_str = "" #
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        # Append examples to the system prompt
                        full_system_prompt = system_prompt + "\n\n" + example_str + "\n\n\n"
                    else:
                        full_system_prompt = system_prompt
                        
                    response = client.generate(model=model,
                        options={'temperature': 0.6, 'num_ctx': 8192, 'top_p': 0.3, 'num_predict': 1024, 'num_gpu': 99},
                        stream=False,
                        system=full_system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    break
                
                except Exception as e:
                    print("Exception on Input: ", e)
                    print("\nCheck example_str same or not: \n",example_str)
                    sys.stdout.flush()
                    if(count<15):
                        continue
                    else:
                        print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                        break 
            try:
                flow_json = json.loads(actual_output)
                
                # The LLM should have generated the "switch" key.
                # If not, we could inject it, but the prompt is strict.
                if "switch" not in flow_json:
                    print(f"Warning: LLM output missing 'switch' key. Injecting DPID {device_id_str}.")
                    flow_json["switch"] = device_id_str

                return flow_json
            
            except Exception as e:
                print(f"Exception found parsing LLM output: {e}")

In [1018]:
def run_LLM_Slice(intent):

    system_prompt = SLICING_PROMPT
    
    for model in my_models_translate_real:     
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                #options={'device': 'cpu'},
                stream=False,
                system=system_prompt,
                prompt=intent,
                format='json'
            )
            
            output = response['response'].strip()
            response_json = json.loads(output)
            
            #print("\nCheckpoint*******Exiting LLM Slicing detection\n\n******")
            return response_json            
        
        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            continue

In [1019]:
def extract_dpid_string_floodlight(intent: str) -> Optional[str]:
    """
    Extracts the integer datapath ID (dpid) and formats it
    as a Floodlight-compatible colon-hex string.
    """
    match = re.search(r'\b(?:switch|dpid|node|device)(?:_id)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        dpid_int = int(match.group(1))
        # Format as 00:00:00:00:00:00:00:0X
        return f"00:00:00:00:00:00:00:{dpid_int:02x}"
    return None

def get_iface_for_port(device_id_int: int, port_no: int | str) -> str:
    """
    Builds the Mininet/OVS interface name (e.g., s4-eth1) from an
    integer DPID (device_id) and port number.
    This is for OVS commands, so it's controller-agnostic.
    """
    switch_name = f"s{device_id_int}"
    return f"{switch_name}-eth{port_no}"

def classify_floodlight_flow_rule(flow_rule: dict):
    """
    Classify a Floodlight static flow rule into a type and compute its specificity.
    This version handles the flat JSON structure.
    
    Returns: (rule_type: str, specificity: float)
    """
    
    # --- 1. Rule Type Detection ---
    # Actions are a single comma-separated string, or absent for DROP
    actions_str = flow_rule.get("actions")
    rule_type = "unknown"

    if actions_str is None:
        # actions key is absent
        rule_type = "security" 
    elif "set_queue" in actions_str:
        rule_type = "qos"
    elif "output" in actions_str:
        rule_type = "forwarding"
    else:
        # Includes empty string "" which is also a drop
        rule_type = "security"

    # --- 2. Specificity Computation ---
    specificity = 0.0
    
    # Define keys that are *not* match fields
    control_keys = {"switch", "name", "active", "priority", "actions"}

    for key, value in flow_rule.items():
        if key in control_keys:
            continue
        
        # This is a match key
        specificity += 1.0 
        
        if key in ("ipv4_src", "ipv4_dst", "ipv6_src", "ipv6_dst") and value:
            try:
                ip_net = ipaddress.ip_network(value, strict=False) 
                if "v6" in key:
                    specificity += (ip_net.prefixlen / 128.0)
                else:
                    specificity += (ip_net.prefixlen / 32.0)
            except Exception:
                pass 
                
    return rule_type, specificity

def resolve_floodlight_conflict(rule1, rule2):
    """
    Resolve conflict between two Floodlight rules using Type > Specificity > Priority.
    Returns: winner_rule, loser_rule
    """
    type_priority = {"security": 3, "qos": 2, "forwarding": 1, "unknown": 0}

    type1, spec1 = classify_floodlight_flow_rule(rule1)
    type2, spec2 = classify_floodlight_flow_rule(rule2)

    p1 = int(rule1.get("priority", 0))
    p2 = int(rule2.get("priority", 0))

    # --- Resolution Hierarchy ---
    
    # 1. Type
    type1_score = type_priority.get(type1, 0)
    type2_score = type_priority.get(type2, 0)
    
    if type1_score > type2_score:
        return rule1, rule2
    elif type2_score > type1_score:
        return rule2, rule1

    # 2. Specificity
    if spec1 > spec2:
        return rule1, rule2
    elif spec2 > spec1:
        return rule2, rule1

    # 3. Priority
    if p1 > p2:
        return rule1, rule2
    elif p2 > p1:
        return rule2, rule1

    # Tie
    return None, None

def adjust_priority_floodlight(winner_rule: dict, loser_rule: dict, step: int = 10) -> dict:
    """
    Adjusts the priority of the winning Floodlight flow rule.
    Priority is a string in Floodlight, but we treat it as int.
    """
    loser_priority = int(loser_rule.get("priority", 0))
    winner_priority = int(winner_rule.get("priority", 0))

    new_priority = max(winner_priority, loser_priority + step)
    
    if new_priority > 65535:
        print(f"Warning: Calculated priority ({new_priority}) exceeds max (65535). Capping at 65535.")
        new_priority = 65535

    # Store it as a string, as required by the prompt
    winner_rule["priority"] = str(new_priority)
    return winner_rule

def extract_inner_flow(rule):
    # This function is no longer needed for Floodlight's flat JSON,
    # but we keep it to avoid breaking end_to_end_IBN
    return rule

def normalize_ip_prefix(ip_str):
    """Converts '10.0.1.1' to '10.0.1.1/32'."""
    if not ip_str:
        return None
    ip_str = str(ip_str)
    if "/" not in ip_str:
        try:
            ipaddress.ip_address(ip_str)
            return f"{ip_str}/32"
        except ValueError:
            return ip_str 
    return ip_str

def normalize_numeric_field(field_str):
    """Converts '0x800', '0x11', '17', '80' to a standard integer."""
    if not field_str:
        return None
    try:
        # int(str, 0) handles hex and decimal
        return int(str(field_str), 0)
    except (ValueError, TypeError):
        return str(field_str) # Fallback

def get_operational_flow_by_match(dpid_str: str, static_flow_rule: dict):
    """
    Finds the *operational* flow (with stats) that matches the
    match fields of a *static* flow rule.
    """
    op_flows = get_operational_flows_floodlight(dpid_str)
    
    # --- 1. Build a NORMALIZED Target Match from Static Rule ---
    control_keys = {"switch", "name", "active", "priority", "actions"}
    target_match_normalized = {}
    
    for k, v in static_flow_rule.items():
        if k in control_keys:
            continue
        
        # Normalize the target values *before* comparison
        if k in ("ipv4_src", "ipv4_dst", "arp_spa", "arp_tpa"):
            target_match_normalized[k] = normalize_ip_prefix(v)
        elif k in ("eth_src", "eth_dst", "arp_sha", "arp_tha"):
            target_match_normalized[k] = str(v).lower()
        else:
            # This handles eth_type, ip_proto, ports, etc.
            target_match_normalized[k] = normalize_numeric_field(v)
            
    target_prio = int(static_flow_rule.get("priority", 0))

    if not op_flows:
        print("[WARN] get_operational_flow_by_match: Received no operational flows from switch.")
        return None

    # --- 2. Iterate and Compare ---
    for op_flow in op_flows:
        
        op_prio_int = int(op_flow.get("priority", -1))
        if op_prio_int != target_prio:
            continue
            
        op_match = op_flow.get("match", {})
        
        # 2a. Check for exact length match
        if len(op_match) != len(target_match_normalized):
            continue
            
        # 2b. Build a NORMALIZED Operational Match
        op_match_normalized = {}
        for k, v in op_match.items():
            if k in ("ipv4_src", "ipv4_dst", "arp_spa", "arp_tpa"):
                op_match_normalized[k] = normalize_ip_prefix(v)
            elif k in ("eth_src", "eth_dst", "arp_sha", "arp_tha"):
                op_match_normalized[k] = str(v).lower()
            else:
                op_match_normalized[k] = normalize_numeric_field(v)
        
        # 2c. Compare the two normalized dictionaries
        if op_match_normalized == target_match_normalized:
            return op_flow # Found it!
            
    # --- 3. No Match Found ---
    return None

In [1020]:
def end_to_end_IBN(intent):

    # 1. Extract DPID
    device_dpid_str = extract_dpid_string_floodlight(intent)

    if device_dpid_str is None:
        print("Error: Could not determine switch DPID from intent.")
        return False, None, None, None, None

    global switch_id_for_llm_assurance 
    # Store the integer part for OVS helpers
    switch_id_for_llm_assurance = int(device_dpid_str.split(":")[-1], 16)

    # 2. Translate Intent
    # The LLM will generate the full JSON, including "switch" and "name"
    intent_JSON = run_LLM_IBN(intent, device_dpid_str)
    
    if not intent_JSON or "switch" not in intent_JSON or "name" not in intent_JSON:
         print("Error: LLM translation failed or JSON is malformed (missing 'switch' or 'name').")
         return False, None, None, None, None
         
    # The DPID from the JSON is the source of truth
    device_dpid_str = intent_JSON["switch"]

    # Get the base name from the LLM
    base_name = intent_JSON.get("name", "flow-unnamed")
    # Create a unique suffix (e.g., 'a1b2c3d4')
    unique_suffix = secrets.token_hex(4)
    # Create the new, guaranteed-unique name
    flow_name = f"{base_name}-{unique_suffix}"
    
    # Update the JSON object with the new unique name before proceeding
    intent_JSON["name"] = flow_name

    global gl_flow_name
    gl_flow_name = flow_name

    # 3. Conflict Detection
    conflict_status, conflict_details, which_flow_conflict = conflict(device_dpid_str, intent_JSON)

    if((conflict_status != 1) and (conflict_status != 0)):
        print("\nCheck Conflict Detection Module, LLM did not produce a valid JSON for conflict detection.\n")
        return False, None, None, None, None

    elif (conflict_status == 1):
        print("Conflict detected. Resolving...")
        winner, non_winner = resolve_floodlight_conflict(intent_JSON, which_flow_conflict)

        if winner is None:
            print("\nConflict resolution resulted in a tie.")
            print(f"\nThe New flow rule:\n{intent_JSON}\nConflicts with existing rule:\n{which_flow_conflict}")
            print(f"\nExisting Flow Rule Location: In switch: {device_dpid_str}\nConflict Details: {conflict_details}")
            return False, "Tie", None, None, None
            
        elif winner.get("name") != flow_name:
            # The existing rule won
            print(f"\nExisting Flow Rule that Conflicts (and won):\n{winner}")
            print(f"\nThe New Flow Rule Attempted (and lost):\n{non_winner}")
            print(f"\nConflict Details : {conflict_details}")
            return False, "existing_rule_win", None, None, None
            
        else:
            # The new rule won, needs priority adjustment
            print(f"Conflict Resolved. Winner Flow Rule (New):\n{winner}")
            print(f"\nShadowed Flow Rule (Existing):\n{non_winner}")
            print(f"\nConflict Details : {conflict_details}")
            updated_flow_json = adjust_priority_floodlight(winner, non_winner)
            intent_JSON = updated_flow_json   

    # 4. Install Flow
    try:
        success = add_flow_floodlight(intent_JSON)
        if not success:
            raise Exception("add_flow_floodlight returned False")
        print(f"Successfully pushed flow '{flow_name}' to dpid: {device_dpid_str}")

    except Exception as e:
            print(f"Exception found while installing flow rule: {e}")
            sys.stdout.flush()
            return False, None, None, None, None
    
    # 5. Verify Installation
    try:
        time.sleep(.5) # Give controller time to install
        
        # Use the new exists_floodlight() helper
        verification_status, returned_static_flow = exists_floodlight(
            device_dpid_str, 
            flow_name
        )
        
        if(verification_status == True):
            time.sleep(.5)
            # --- NEW: Fetch the operational rule ---
            #print("Now, fetching corresponding operational flow rule for assurance...")
            operational_rule_with_stats = get_operational_flow_by_match(
                device_dpid_str, 
                intent_JSON # Use the rule we pushed to match against
            )
            
            if not operational_rule_with_stats:
                print("[WARN] Static rule found, but matching operational rule not found on switch yet.")
                # We'll return the static rule as a fallback
                operational_rule_with_stats = intent_JSON 
            else:
                print("Successfully fetched operational flow rule.")
            # --- END NEW ---

            flow_id = flow_name
            
            # The 'operational_flow_rule' from static list is what we *pushed*.
            # For assurance, we need the *operational* flow with stats.
            # We'll fetch that in the assurance step.
            # For now, return the *intended* flow rule.
            
            # Get the integer dpid for OVS helpers
            dpid_int = int(device_dpid_str.split(":")[-1], 16)
            
            return True, flow_id, dpid_int, intent_JSON, operational_rule_with_stats
        else:
             print("Flow verification failed. Rule not found in switch's static flow list.")
             return False, None, None, None, None

    except Exception as e:
            print(f"Exception found while verifying flow rule: {e}")
            sys.stdout.flush()
            return False, None, None, None, None

In [1021]:
def generate_corrective_action_prompt_floodlight(intent_nl, operational_flow_rule, dpid_str, ping_count,
                                                 candidate_src_ip, candidate_dst_ip, ping_output):
    """
    Generates a prompt for the LLM to suggest corrective actions for a FAILED security intent.
    This version is adapted for Floodlight's flat flow format.
    'operational_flow_rule' is the rule that was pushed (static format).
    """
    # Get existing operational flows from the switch
    # Note: these are in OPERATIONAL format (with 'match' objects)
    # This might be confusing for the LLM. Let's send the STATIC flows.
    existing_flows = get_static_flows_floodlight(dpid_str)

    prompt_sections = []
    prompt_sections.append(
        "You are an SDN network assistant. Your task is to recommend a ranked list of corrective actions "
        "to enforce a **security intent** that failed during assurance testing."
    )
    prompt_sections.append(f"1. **Security Intent (in Natural Language)**:\n{intent_nl}")

    # Use Floodlight's flow format
    prompt_sections.append("2. **Floodlight Flow Rule for the Security Intent (Pushed to Switch)**:")
    prompt_sections.append(json.dumps(operational_flow_rule, indent=2))

    prompt_sections.append("3. **Existing Static Flow Rules in the Same Switch (DPID)**:")
    prompt_sections.append(json.dumps(existing_flows, separators=(",", ":")))

    prompt_sections.append(
        f"""4. **Assurance Test Result**:
        - Ping Source IP: {candidate_src_ip}
        - Ping Destination IP: {candidate_dst_ip}
        - Ping Count: {ping_count}
        - Ping Output:
        {ping_output}"""
    )
    
    # Instructions updated for Floodlight's JSON structure
    prompt_sections.append(
        "---\nNow, based on the above data, generate a ranked list of corrective actions. "
        "You must use only the following predefined corrective actions:\n"
        "1. Correct Match Fields\n"
        "2. Increase Priority\n"
        "3. Fix Action Field\n\n"
        "For each action:\n"
        "- **For 'Correct Match Fields'**: Specify *exactly* which top-level key-value pairs should be set (e.g., \"ipv4_src\": \"10.0.1.1\", \"eth_type\": \"0x0800\"), and which keys (if any) should be removed.\n"
        "- **For 'Increase Priority'**: Indicate which existing rule(s) (by 'name') are overshadowing the candidate rule, their current priority value(s), and the exact priority value the candidate rule should be set to.\n"
        "- **For 'Fix Action Field'**: Only include this if the 'actions' key is present. The suggestion should be to REMOVE the 'actions' key entirely.\n\n"
        "Rank the actions and explain your reasoning.\n"
        "Return your answer ONLY in the following strict JSON format:\n"
        "{\n"
        "  \"recommended_actions\": [\n"
        "    {\n"
        "      \"rank\": 1,\n"
        "      \"action\": \"Correct Match Fields\",\n"
        "      \"suggestion\": {\n"
        "        \"set_fields\": {\"ipv4_src\": \"10.0.0.1\", \"ipv4_dst\": \"10.0.0.2\"},\n"
        "        \"remove_fields\": [\"tcp_dst\"],\n"
        "        \"reasoning\": \"The match fields do not match the intended source and destination.\"\n"
        "      }\n"
        "    },\n"
        "    {\n"
        "      \"rank\": 2,\n"
        "      \"action\": \"Increase Priority\",\n"
        "      \"suggestion\": {\n"
        "        \"conflicting_rules\": [ {\"name\": \"existing-flow-1\", \"priority\": \"400\"} ],\n"
        "        \"recommended_priority\": 410,\n"
        "        \"reasoning\": \"The candidate rule is overshadowed by a rule with higher priority.\"\n"
        "      }\n"
        "    }\n"
        "  ]\n"
        "}\n"
        "Omit any action that is not relevant."
    )
    return "\n\n".join(prompt_sections)

In [1022]:
def Run_assurance_LLM (assurance_prompt):
    """
    (This function is controller-agnostic and does not need changes.)
    Sends the prompt to the LLM and gets a JSON response.
    """
    for model in my_models_conflict_real:    
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                stream=False,
                system="",
                prompt=assurance_prompt,
                format='json'
            )
            output = response['response'].strip()
            response_json = json.loads(output)
            return response_json            
        except Exception as e:
            print(f"Exception found at assurance LLM for corrective action generation: {e}")
            sys.stdout.flush()
            continue
    return None # Return None if all models fail

In [1023]:
def execute_command_full(command, timeout=15, with_sudo=False):
    # Run the command as-is or under sudo (single layer)
    cmd = command
    if with_sudo:
        # -S: read password from stdin; -p '' suppresses the prompt text
        cmd = f"sudo -S -p '' {command}"
        cmd = f"printf '%s\\n' {shlex.quote(sudo_password)} | {cmd}"
    try:
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return res.stdout, res.stderr
    except subprocess.TimeoutExpired as e:
        print(f"Command timed out: {command}")
        stdout_str = e.stdout if isinstance(e.stdout, str) else (e.stdout.decode('utf-8', 'ignore') if e.stdout else "")
        stderr_str = e.stderr if isinstance(e.stderr, str) else (e.stderr.decode('utf-8', 'ignore') if e.stderr else "Timeout")
        return stdout_str, stderr_str

def get_mininet_host_pid(src_host):
    """
    Robustly get the PID of a Mininet host process (e.g., 'h1' or 'h1onos') regardless of user.
    Looks for a process with command containing 'mininet:<src_host>'.
    """
    try:
        ps_output = subprocess.check_output(["ps", "-eo", "pid,args"], text=True).strip().splitlines()
    except subprocess.CalledProcessError as e:
        raise RuntimeError("Failed to run 'ps -eo pid,args'") from e

    for line in ps_output:
        if f"mininet:{src_host}" in line and "grep" not in line:
            parts = line.strip().split(None, 1)
            if len(parts) == 2:
                pid_str, cmd = parts
                try:
                    pid = int(pid_str)
                    return pid
                except ValueError:
                    continue
    raise RuntimeError(f"No Mininet host process found for '{src_host}'.")

def extract_host_and_ip_floodlight(flow_data: dict):
    """
    Extracts source and destination IPs from a Floodlight static flow rule (flat format).
    Returns: (src_host, dst_host, src_ip, dst_ip)
    """
    src_ip = dst_ip = None
    
    # Match keys are at the top level
    if "ipv4_src" in flow_data:
        src_ip = flow_data["ipv4_src"].split("/")[0]
    if "ipv4_dst" in flow_data:
        dst_ip = flow_data["ipv4_dst"].split("/")[0]

    src_host = ip_to_host.get(src_ip) if src_ip else None
    dst_host = ip_to_host.get(dst_ip) if dst_ip else None

    return src_host, dst_host, src_ip, dst_ip

def floodlight_assurance_for_security_intent(src_ip, dst_ip, rule_dpid_int, ping_count=2):
    """
    Tests the data plane to assure a security (block) intent.
    'rule_dpid_int' is the integer DPID of the switch where the rule lives.
    """
    all_ips = ["10.0.1.1", "10.0.1.2", "10.0.1.3", "10.0.1.4"]
    global llm_caller_flag
    llm_caller_flag = 0

    def perform_ping(source_ip, target_ip):
        source_host = ip_to_host.get(source_ip)
        if not source_host:
            print(f"[WARNING] Unknown source host for IP: {source_ip}")
            return ""
        try:
            host_pid = get_mininet_host_pid(source_host)
        except Exception as e:
            print(f"[ERROR] Cannot get PID for host {source_host}: {e}")
            return ""
        ping_cmd = f"mnexec -a {host_pid} ping -c {ping_count} {target_ip}"
        out, err = execute_command_full(ping_cmd, with_sudo=True)
        return f"{out or ''}{err or ''}"

    # Case 1: src and dst IP both present
    if src_ip and dst_ip:
        print(f"[INFO] Testing {src_ip} to {dst_ip}")
        output = perform_ping(src_ip, dst_ip)
        if output and "100% packet loss" in output:
            print(f"[PASS] Intent effective. Traffic from {src_ip} to {dst_ip} is blocked.")
        elif output and "0% packet loss" in output:
            print(f"[FAIL] Intent NOT effective. Traffic from {src_ip} to {dst_ip} is NOT blocked.")
            llm_caller_flag = 1
        else:
            print("[WARN] Inconclusive result. Ping output:\n", output, "\nPlease check first why ping is not working.")
            llm_caller_flag = 2
        return ping_count, src_ip, dst_ip, output
    
    # Case 2: src_ip is None → test all src IPs → dst_ip
    elif src_ip is None and dst_ip:
        print(f"[INFO] Testing multiple sources to {dst_ip}")
        candidate_src = "" # Keep scope
        for candidate_src in all_ips:
            if candidate_src == dst_ip:
                continue
            output = perform_ping(candidate_src, dst_ip)
            if output and "100% packet loss" in output:
                continue
            elif output and "0% packet loss" in output:
                print(f"[FAIL] Intent NOT effective. {candidate_src} → {dst_ip} was NOT blocked.")
                llm_caller_flag = 1
                return ping_count, candidate_src, dst_ip, output
            else:
                print(f"[WARN] Inconclusive result from {candidate_src} → {dst_ip}:\n{output}", "\nPlease check first why ping is not working.")
                llm_caller_flag = 2
                return ping_count, candidate_src, dst_ip, output
        print(f"[PASS] Intent effective. All sources blocked from reaching {dst_ip}.")
        return ping_count, candidate_src, dst_ip, "" # Return last output (which was empty)

    # Case 3: dst_ip is None → test src_ip → all destinations
    elif dst_ip is None and src_ip:
        print(f"[INFO] Testing {src_ip} for multiple destinations")
        candidate_dst = "" # Keep scope
        for candidate_dst in all_ips:
            if candidate_dst == src_ip:
                continue
            try:
                # Use the colon-hex DPID formats from SW_OF
                src_attach_dev_str, _ = HOST_ATTACH.get(src_ip, (None, None))
                dst_attach_dev_str, _ = HOST_ATTACH.get(candidate_dst, (None, None))
                
                # Convert the integer rule_dpid (e.g., 2) to its string key ("2")
                rule_dev_str = SW_OF.get(str(rule_dpid_int))

                if (src_attach_dev_str and 
                    src_attach_dev_str == dst_attach_dev_str and 
                    src_attach_dev_str != rule_dev_str):
                    
                    print(f"[INFO] Skipping test {src_ip} -> {candidate_dst} (local on {src_attach_dev_str}, rule is on {rule_dev_str})")
                    continue
            except Exception as e:
                print(f"[WARN] Topology check failed: {e}. Proceeding with test.")
            
            output = perform_ping(src_ip, candidate_dst)
            if output and "100% packet loss" in output:
                continue
            elif output and "0% packet loss" in output:
                print(f"[FAIL] Intent NOT effective. {src_ip} → {candidate_dst} was NOT blocked.")
                llm_caller_flag = 1
                return ping_count, src_ip, candidate_dst, output
            else:
                print(f"[WARN] Inconclusive result from {src_ip} → {candidate_dst}:\n{output}", "\nPlease check first why ping is not working.")
                llm_caller_flag = 2
                return ping_count, src_ip, candidate_dst, output
        print(f"[PASS] Intent effective.")
        return ping_count, src_ip, candidate_dst, ""
    
    else:
        print("[ERROR] Both source and destination IPs are missing. Cannot evaluate intent.")
        llm_caller_flag = 2
        return ping_count, None, None, None

def correct_match_fields_floodlight(candidate_flow, set_fields, remove_fields):
    """
    Update candidate_flow's top-level match fields for Floodlight.
    """
    # Remove fields
    for field in remove_fields:
        candidate_flow.pop(field, None)
    
    # Set/update fields
    candidate_flow.update(set_fields)
    
    return candidate_flow

def increase_priority_floodlight(candidate_flow, recommended_priority):
    # Ensure priority is stored as a string
    candidate_flow['priority'] = str(recommended_priority)
    return candidate_flow

def fix_action_field_floodlight(candidate_flow):
    """
    Make sure 'actions' key is REMOVED (for a block rule).
    """
    candidate_flow.pop("actions", None)
    return candidate_flow

def parse_and_execute_corrective_actions_floodlight(candidate_flow, llm_response, dpid_int):
    """
    Applies corrective actions suggested by the LLM for Floodlight.
    'candidate_flow' is the static flow rule that failed verification.
    'dpid_int' is the integer DPID.
    """
    if not llm_response or "recommended_actions" not in llm_response:
        print("LLM provided no valid corrective actions.")
        return False
        
    actions = sorted(llm_response["recommended_actions"], key=lambda x: x["rank"])
    
    # We need the original src/dst for re-verification
    _, _, src_ip, dst_ip = extract_host_and_ip_floodlight(candidate_flow)
    
    # We need the flow's name to delete/update it
    original_flow_name = candidate_flow.get("name")
    if not original_flow_name:
        print("Error: Cannot execute corrective action on flow with no 'name'.")
        return False

    print("Deleting the original, failed flow rule...")
    delete_flow_floodlight(original_flow_name)
    
    for action_item in actions:
        action = action_item["action"]
        suggestion = action_item.get("suggestion", {})
        print(f"Triggering action: {action} (rank {action_item['rank']})")

        corrected_flow = candidate_flow.copy()
        
        if action == "Correct Match Fields":
            set_fields = suggestion.get("set_fields", {})
            remove_fields = suggestion.get("remove_fields", [])
            corrected_flow = correct_match_fields_floodlight(corrected_flow, set_fields, remove_fields)
        
        elif action == "Increase Priority":
            recommended_priority = suggestion.get("recommended_priority")
            if recommended_priority:
                corrected_flow = increase_priority_floodlight(corrected_flow, recommended_priority)
            else:
                print("Warning: LLM suggested priority increase but gave no value.")
                continue
                
        elif action == "Fix Action Field":
            corrected_flow = fix_action_field_floodlight(corrected_flow)
        
        else:
            print(f"Unknown action: {action}")
            continue

        # Ensure the name remains the same
        corrected_flow["name"] = original_flow_name
        
        # 2. Push corrected flow
        print(f"Pushing corrected flow: {json.dumps(corrected_flow)}")
        try:
            add_flow_floodlight(corrected_flow)
            time.sleep(0.5)
        except Exception as e:
            print(f"Exception while pushing corrected flow: {e}")
            continue # Try next action

        # 3. Re-verify
        print("Re-verifying intent...")
        global llm_caller_flag
        # This will ping and set llm_caller_flag (0=PASS, 1=FAIL, 2=WARN)
        floodlight_assurance_for_security_intent(src_ip, dst_ip, dpid_int) 
        
        if llm_caller_flag == 0:
            print(f"Intent deviation resolved after action: {action}")
            return True
        else:
            print(f"Deviation not fixed after action: {action}. Deleting flow and trying next action...")
            delete_flow_floodlight(original_flow_name) # Clean up

    print("\nAll suggested corrective actions exhausted, but deviation remains. Escalate to operator.")
    return False

def Floodlight_assurance_for_qos_intent(src_host, dst_host, src_ip, dst_ip, ping_count=2):
    pass
def Floodlight_assurance_for_forwarding_intent(src_host, dst_host, src_ip, dst_ip, ping_count=2):
    pass

In [1024]:
# QoS verification helper functions (Floodlight-compatible)

# --- OVS/Mininet Helpers (Controller-Agnostic) ---
# (These helpers interact with OVS/Mininet directly)

def _sudo(cmd, timeout=30):
    return execute_command_full(f"bash -lc {shlex.quote(cmd)}", timeout=timeout, with_sudo=True)

def _ns(pid, cmd, timeout=90):
    return execute_command_full(f"mnexec -a {pid} bash -lc {shlex.quote(cmd)}", timeout=timeout, with_sudo=True)

def run_in_host(host_pid: int, cmd: str, timeout: int = 30) -> tuple[str, str]:
    # Helper function to run a command inside a host namespace
    base = f"mnexec -a {host_pid} {cmd}"
    return execute_command_full(base, timeout=timeout, with_sudo=True)

def ensure_no_iperf_server(host_pid: int, port: int) -> None:
    run_in_host(host_pid, f"/usr/bin/pkill -f {shlex.quote(f'iperf3 -s -p {int(port)}')} || true", timeout=5)
    # --- FIX: Corrected escaping for the bash command ---
    run_in_host(
        host_pid,
        f"bash -lc \"/usr/bin/pgrep -af 'iperf3.*-s.*-p {int(port)}' | /usr/bin/awk '{{{{print $1}}}}' | /usr/bin/xargs -r /bin/kill -9\"",
        timeout=5
    )

def start_iperf_server(host_pid: int, port: int, extra_args: str = "") -> None:
    ensure_no_iperf_server(host_pid, port)
    # --- FIX: Corrected escaping for the bash command ---
    run_in_host(
            host_pid,
            f"bash -lc 'nohup /usr/bin/iperf3 -s -p {int(port)} --one-off {extra_args} "
            f">/tmp/iperf3_s_{int(port)}.log 2>&1 & echo $!'",
            timeout=5
        )
    deadline = time.time() + 4.0
    while time.time() < deadline:
        # --- FIX: Corrected escaping for the bash command ---
        out, err = run_in_host(
            host_pid,
            f"bash -lc \"/usr/bin/ss -ltnp | grep ':{int(port)} ' || true\"",
            timeout=3
        )
        if out.strip():
            return
    # --- FIX: Changed tail from '-n +200' (from line 200) to '-n 200' (last 200 lines) ---
    log, _ = run_in_host(
        host_pid,
        f"bash -lc 'tail -n 200 /tmp/iperf3_s_{int(port)}.log 2>/dev/null || true'",
        timeout=3
    )
    raise RuntimeError(f"iperf3 server failed to bind/listen on port {port}. Server log:\n{log}")

    # --- FIX: Corrected escaping for the bash command ---
    # log, _ = run_in_host(
    #     host_pid,
    #     f"bash -lc 'tail -n +200 /tmp/iperf3_s_{int(port)}.log 2>/dev/null || true'",
    #     timeout=3
    # )
    # raise RuntimeError(f"iperf3 server failed to bind/listen on port {port}. Server log:\n{log}")

def stop_iperf_server(host_pid: int, port: int) -> None:
    ensure_no_iperf_server(host_pid, port)

def parse_qos_show(iface: str) -> dict:
    cmd = f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)}"
    stdout_text, stderr_text = execute_command_full(cmd, with_sudo=True)

    if "No QoS configured" in stdout_text or "No QoS configured" in stderr_text:
        return {"raw": stdout_text, "queues": {}}
    if not stdout_text and "Timeout" in stderr_text:
        print(f"[WARN] Command timed out: {cmd}")
        return {"raw": "Timeout", "queues": {}}
    
    raw = stdout_text
    queues = {}
    qos_meta = {"raw": raw}
    current_obj = None
    
    for raw_line in raw.splitlines():
        line = raw_line.strip()
        if not line:
            continue
        header_match = re.match(r"^(QoS|Queue|Default):\s*(.*)", line)
        if header_match:
            obj_type = header_match.group(1)
            if obj_type == "QoS":
                current_obj = qos_meta
            elif obj_type == "Queue":
                q_id = line.split()[1].replace(":", "")
                queues[q_id] = {}
                current_obj = queues[q_id]
            elif obj_type == "Default":
                queues["default"] = {}
                current_obj = queues["default"]
            continue
        
        if current_obj is not None:
            kv_match = re.match(r"^([\w-]+):\s*(\d+)", line)
            if kv_match:
                key = kv_match.group(1)
                try:
                    value = int(kv_match.group(2))
                except ValueError:
                    value = kv_match.group(2)
                current_obj[key] = value

    qos_meta["queues"] = queues
    return qos_meta

def snapshot_queue(iface: str, queue_id: int | str):
    parsed = parse_qos_show(iface)
    q = parsed["queues"].get(str(queue_id)) or parsed["queues"].get("default") or {}
    return {
        "min_rate": q.get("min-rate"),
        "max_rate": q.get("max-rate"),
        "tx_bytes": q.get("tx_bytes"),
        "tx_packets": q.get("tx_packets"),
        "_raw": parsed["raw"][:1200]
    }

def _parse_iperf_sender(line: str) -> tuple[float, int] | tuple[None, None]:
    try:
        # --- FIX: Removed double-escaped backslashes (e.g., \\d -> \d) ---
        match = re.search(r"(\d+(?:\.\d+)?)\s+MBytes\s+(\d+(?:\.\d+)?)\s+Mbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024 * 1024)
            mbps = float(match.group(2))
            return mbps, bytes_sent
        
        match = re.search(r"(\d+(?:\.\d+)?)\s+KBytes\s+(\d+(?:\.\d+)?)\s+Mbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024)
            mbps = float(match.group(2)) # It's already in Mbits/sec
            return mbps, bytes_sent

        match = re.search(r"(\d+(?:\.\d+)?)\s+KBytes\s+(\d+(?:\.\d+)?)\s+Kbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024)
            mbps = float(match.group(2)) / 1024.0
            return mbps, bytes_sent
        
        match = re.search(r"(\d+(?:\.\d+)?)\s+Bytes\s+(\d+(?:\.\d+)?)\s+bits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)))
            mbps = float(match.group(2)) / 1_000_000.0
            return mbps, bytes_sent
    except Exception as e:
        print(f"[WARN] Failed to parse iperf line '{line}': {e}")
        pass
    return None, None

def _parse_iperf_sender_old(line: str) -> tuple[float, int] | tuple[None, None]:
    try:
        match = re.search(r"(\\d+(?:\\.\\d+)?)\\s+MBytes\\s+(\\d+(?:\\.\\d+)?)\\s+Mbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024 * 1024)
            mbps = float(match.group(2))
            return mbps, bytes_sent
        
        # --- THIS IS THE CRITICAL FIX FOR THE PARSER ---
        match = re.search(r"(\\d+(?:\\.\\d+)?)\\s+KBytes\\s+(\\d+(?:\\.\\d+)?)\\s+Mbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024)
            mbps = float(match.group(2)) # It's already in Mbits/sec
            return mbps, bytes_sent
        # --- END OF FIX ---

        match = re.search(r"(\\d+(?:\\.\\d+)?)\\s+KBytes\\s+(\\d+(?:\\.\\d+)?)\\s+Kbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024)
            mbps = float(match.group(2)) / 1024.0
            return mbps, bytes_sent
        match = re.search(r"(\\d+(?:\\.\\d+)?)\\s+Bytes\\s+(\\d+(?:\\.\\d+)?)\\s+bits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)))
            mbps = float(match.group(2)) / 1_000_000.0
            return mbps, bytes_sent
    except Exception as e:
        print(f"[WARN] Failed to parse iperf line '{line}': {e}")
        pass
    return None, None

def _iperf_text_summary(client_pid, dst_ip, dst_port, duration, extra_args="") -> dict:
    iperf_cmd_str = f'/usr/bin/iperf3 -c {dst_ip} -p {dst_port} -t {duration} {extra_args}'
    cmd = f"mnexec -a {client_pid} bash -lc {shlex.quote(iperf_cmd_str)}"
    print(f"\n[DEBUG] Running iperf command: {cmd}\n") # <-- ADDED DEBUG
    stdout_text, stderr_text = execute_command_full(cmd, timeout=max(15, duration + 10), with_sudo=True)
    
    if "Connection refused" in stderr_text:
        print("[DEBUG] iperf stdout (raw):", stdout_text) # <-- ADDED DEBUG
        print("[DEBUG] iperf stderr (raw):", stderr_text) # <-- ADDED DEBUG
        raise RuntimeError(f"Iperf connection refused. Is server running on {dst_ip}:{dst_port}?")
    if "No route to host" in stderr_text:
        print("[DEBUG] iperf stdout (raw):", stdout_text) # <-- ADDED DEBUG
        print("[DEBUG] iperf stderr (raw):", stderr_text) # <-- ADDED DEBUG
        raise RuntimeError(f"Iperf 'No route to host' for {dst_ip}. Check pinning flows.")

    stdout_lines = stdout_text.splitlines()
    target_line = None
    for ln in reversed(stdout_lines):
        if ("bits/sec" in ln and 
           (f" 0.0-{duration:.1f}" in ln or f" 0.00-{duration:.02f}" in ln or " 0.0-" in ln)):
            if ln.strip().startswith("[SUM]"):
                target_line = ln
                break
            # This regex is safer for lines like "[ 5]"
            if re.match(r'^\\s*\\[\\s*\\d+\\]', ln.strip()):
                target_line = ln
            elif ln.strip().startswith("[") and "sender" in ln: # Fallback for older iperf
                target_line = ln
    
    if not target_line:
        # --- THIS IS THE MOST IMPORTANT PART ---
        # If parsing fails, print everything so we can see the error
        print("\n[DEBUG] --- IPERF SUMMARY PARSING FAILED ---")
        print(f"[DEBUG] iperf stdout (raw):\n{stdout_text}")
        print(f"[DEBUG] iperf stderr (raw):\n{stderr_text}\n")
        # We will return None instead of raising an error
        # to allow the test to fail gracefully.
        return {"sender_mbps": None, "bytes_sent": None, "sender_line": None}

    mbps, bytes_sent = _parse_iperf_sender(target_line)
    return {"sender_mbps": mbps, "bytes_sent": bytes_sent, "sender_line": target_line}


def _iperf_text_summary_old(client_pid, dst_ip, dst_port, duration, extra_args="") -> dict:
    iperf_cmd_str = f'/usr/bin/iperf3 -c {dst_ip} -p {dst_port} -t {duration} {extra_args}'
    cmd = f"mnexec -a {client_pid} bash -lc {shlex.quote(iperf_cmd_str)}"
    stdout_text, stderr_text = execute_command_full(cmd, timeout=max(15, duration + 10), with_sudo=True)
    
    if "Connection refused" in stderr_text:
        raise RuntimeError(f"Iperf connection refused. Is server running on {dst_ip}:{dst_port}?")
    if "No route to host" in stderr_text:
        raise RuntimeError(f"Iperf 'No route to host' for {dst_ip}. Check pinning flows.")

    stdout_lines = stdout_text.splitlines()
    target_line = None
    for ln in reversed(stdout_lines):
        if ("bits/sec" in ln and 
           (f" 0.0-{duration:.1f}" in ln or f" 0.00-{duration:.02f}" in ln or " 0.0-" in ln)):
            if ln.strip().startswith("[SUM]"):
                target_line = ln
                break
            # This regex is safer for lines like "[ 5]"
            if re.match(r'^\s*\[\s*\d+\]', ln.strip()):
                target_line = ln
            elif ln.strip().startswith("[") and "sender" in ln: # Fallback for older iperf
                target_line = ln
    
    if not target_line:
        print("[DEBUG] iperf stdout:", stdout_text)
        print("[DEBUG] iperf stderr:", stderr_text)
        raise RuntimeError("Could not find iperf sender summary line. iperf output was not as expected.")

    mbps, bytes_sent = _parse_iperf_sender(target_line)
    return {"sender_mbps": mbps, "bytes_sent": bytes_sent, "sender_line": target_line}

def ensure_qos_cap(device_id_int: int, port_no: int, qid: int, min_bps: int, max_bps: int, port_cap_bps: int = 100_000_000):
    iface = get_iface_for_port(device_id_int, port_no)
    print(f"[INFO] Setting QoS on {iface}: q{qid} min={min_bps} max={max_bps}, root max-rate={port_cap_bps}bps")
    _sudo(f"ovs-vsctl --if-exists clear Port {shlex.quote(iface)} qos")
    cmd = (
        "ovs-vsctl "
        f"-- --id=@q create Queue other-config:min-rate={min_bps} other-config:max-rate={max_bps} "
        f"-- --id=@qos create QoS type=linux-htb other-config:max-rate={port_cap_bps} queues:{qid}=@q "
        f"-- set Port {shlex.quote(iface)} qos=@qos"
    )
    print(_sudo(cmd))
    print(_sudo(f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)}"))

def _floodlight_flow_counters(flow_json):
    """
    Extracts packet and byte counters from a Floodlight *operational* flow rule.
    """
    if flow_json:
        # FIX: Changed keys to match Floodlight's operational flow output
        return int(flow_json.get("packet_count", 0)), int(flow_json.get("byte_count", 0))
    return None, None

def _floodlight_flow_counters_old(flow_json):
    """
    Extracts packet and byte counters from a Floodlight *operational* flow rule.
    """
    if flow_json:
        return int(flow_json.get("packetCount", 0)), int(flow_json.get("byteCount", 0))
    return None, None

# --- Main QoS Verifier ---

def verify_qos_flow_with_iperf(
    flow_device_dpid_str: str, flow_name: str, # <-- Changed
    queue_device_dpid_int: int, queue_port_no: int, queue_id: int,
    src_ip: str, dst_ip: str, dst_port: int,
    target_mbps: float,
    main_static_flow_rule: dict, # <-- Added this
    duration_sec: int = 8, parallel: int = 8, tcp_mss: int = 1200,
    tolerance_pct: float = 10.0,
    pin_path_flows: list | None = None,
    protocol: str = "tcp",
    udp_bw_mbps: float = 50.0,
    udp_len_bytes: int = 1200,
):
    """
    Verifies a QoS flow using iperf.
    This version is adapted for Floodlight (uses dpid_str and flow_name).
    """
    
    # Resolve actors (OVS/Mininet layer)
    iface = get_iface_for_port(queue_device_dpid_int, queue_port_no)
    client_pid = get_mininet_host_pid(ip_to_host[src_ip])
    server_pid = get_mininet_host_pid(ip_to_host[dst_ip])
    proto = protocol.lower()
    
    # if proto == "tcp":
    #     extra_args = f"-P {parallel} -M {tcp_mss}"
    # else:
    #     #extra_args = f"-u -b {udp_bw_mbps}M -l {udp_len_bytes} -P {max(1, parallel)}"
    #     offer_mbps = max(target_mbps * 1.25, 6.0)  # >= 25% headroom, min 6 Mbps
    #     extra_args = f"-u -b {offer_mbps}M -l {udp_len_bytes} -P 1"

    if proto == "tcp":
        extra_args = f"-P {parallel} -M {tcp_mss}"
    else:
        offer_mbps = max(target_mbps * 1.25, 6.0)  # >= 25% headroom, min 6 Mbps
        
        # --- FIX: Convert bandwidth to an integer in bits per second ---
        # This avoids iperf3 misinterpreting "6.0M"
        offer_bps = int(offer_mbps * 1_000_000)
        extra_args = f"-u -b {offer_bps} -l {udp_len_bytes} -P 1"
        # --- END OF FIX ---

    # 1. Install Pinning Flows
    pushed_pin_flows = [] # This will store the actual flow dicts
    if pin_path_flows:
        pushed_pin_flows = install_pins_from_plan_floodlight(
            pins=pin_path_flows,
            src_ip=src_ip, dst_ip=dst_ip,
            dst_port=dst_port, protocol=protocol
        )
        print(f"Installed {len(pushed_pin_flows)} pinning flows.")

        # --- FIX: Install TCP Helper Flow for iperf3 Control Channel ---
        # This flow matches the TCP control packets and forwards them
        # on the same path as the QoS rule, but to the default queue.
        tcp_helper_flow = {}
        try:
            # We build the match based on the main QoS flow
            # We need all match fields EXCEPT the L4 proto ones (ip_proto, udp_dst)
            control_keys = {"switch", "name", "active", "priority", "actions", "ip_proto", "udp_dst", "tcp_dst"}
            
            # Copy all L2/L3 match fields (like in_port, eth_type, etc.)
            for k, v in main_static_flow_rule.items():
                if k not in control_keys:
                    tcp_helper_flow[k] = v
            
            # Add the TCP-specific L4 match fields
            tcp_helper_flow["ip_proto"] = "6" # TCP
            tcp_helper_flow["tcp_dst"] = str(dst_port)
            
            # Add the control fields
            tcp_helper_flow["switch"] = flow_device_dpid_str
            tcp_helper_flow["priority"] = main_static_flow_rule.get("priority", "200") # Same priority
            tcp_helper_flow["active"] = "true"
            tcp_helper_flow["name"] = f"pin-tcp-helper-{secrets.token_hex(4)}"
            
            # Find the output action, but remove the set_queue action
            original_actions = main_static_flow_rule.get("actions", "").split(',')
            new_actions = [a for a in original_actions if "output=" in a]
            tcp_helper_flow["actions"] = ",".join(new_actions)

            if "output=" in tcp_helper_flow["actions"]:
                print(f"[DEBUG] Installing TCP helper flow: {tcp_helper_flow['name']}")
                add_flow_floodlight(tcp_helper_flow)
                pushed_pin_flows.append(tcp_helper_flow) # Add to cleanup list
            else:
                print("[WARN] Could not create TCP helper flow, 'actions' missing output.")

        except Exception as e:
            print(f"[WARN] Failed to install TCP helper flow: {e}")
        # --- END OF FIX ---

    # --- FIX: Main try block STARTS here ---
    try:
        try:
            _sudo(f"ethtool -K {shlex.quote(iface)} gro off gso off tso off", timeout=6)
        except Exception: 
            pass # This command is non-critical

        # 2. Snapshots before
        flow_before = get_operational_flow_by_match(flow_device_dpid_str, main_static_flow_rule)
        f_pkts0, f_bytes0 = _floodlight_flow_counters(flow_before)
        q0 = snapshot_queue(iface, queue_id)

        # 3. Run iperf
        #start_iperf_server(server_pid, dst_port)
        #server_extra_args = "-u" if proto == "udp" else ""
        #start_iperf_server(server_pid, dst_port, extra_args=server_extra_args)
        start_iperf_server(server_pid, dst_port)

        time.sleep(0.6)
        t0 = time.time()
        iptxt = _iperf_text_summary(client_pid, dst_ip, dst_port, duration_sec, extra_args=extra_args)
        t1 = time.time()
        stop_iperf_server(server_pid, dst_port)
        time.sleep(0.8)

        # 4. Snapshots after
        flow_after = get_operational_flow_by_match(flow_device_dpid_str, main_static_flow_rule)
        f_pkts1, f_bytes1 = _floodlight_flow_counters(flow_after)
        q1 = snapshot_queue(iface, queue_id)

        # 5. Deltas
        elapsed = max(0.001, t1 - t0)
        q_bytes = None if (q0.get("tx_bytes") is None or q1.get("tx_bytes") is None) else (q1["tx_bytes"] - q0["tx_bytes"])
        q_mbps  = (q_bytes * 8 / elapsed / 1e6) if q_bytes is not None else None
        f_pkts_delta = (f_pkts1 - f_pkts0) if (f_pkts0 is not None and f_pkts1 is not None) else None
        f_bytes_delta = (f_bytes1 - f_bytes0) if (f_bytes0 is not None and f_bytes1 is not None) else None
        f_mbps = (f_bytes_delta * 8 / elapsed / 1e6) if (f_bytes_delta is not None) else None

        # 6. Decision
        rate_ok   = (q_mbps is not None) and (abs(q_mbps - target_mbps) <= (tolerance_pct/100.0)*target_mbps)
        packets_ok= (f_pkts_delta or 0) > 0
        verdict   = "PASS" if (rate_ok and packets_ok) else "FAIL"

        # 7. Print summary
        print("\n=== QoS FLOW VERIFICATION ===")
        print(f"Flow (name:{flow_name}) @ DPID {flow_device_dpid_str}  → selector should match {protocol} dst {dst_port} to {dst_ip}")
        print(f"Egress iface={iface} queue={queue_id}  target≈{target_mbps:.3f} Mbps  tol=±{tolerance_pct:.0f}%")
        iperf_mbps_str = f"{iptxt['sender_mbps']:.3f}" if iptxt['sender_mbps'] is not None else "N/A"
        iperf_bytes_str = f"{iptxt['bytes_sent']}" if iptxt['bytes_sent'] is not None else "N/A"
        print(f"iperf sender: {iperf_mbps_str} Mbps  bytes≈{iperf_bytes_str}")
        print(f"Queue Δbytes={q_bytes} over {elapsed:.3f}s  → queue_measured≈{(q_mbps or 0):.3f} Mbps")
        print(f"Flow Δ: packets={f_pkts_delta} bytes={f_bytes_delta}  (flow_measured≈{(f_mbps or 0):.3f} Mbps)")
        print(f"Queue caps (min/max): {q0.get('min_rate')} / {q0.get('max_rate')}  →  {q1.get('min_rate')} / {q1.get('max_rate')}")
        print("VERDICT:", verdict)
        
        if(f_pkts_delta is None):
            print("[WARN] Could not find operational flow to read stats. Verification may be unreliable.")

        # --- FIX: Return statement is now *inside* the try block ---
        return {
            "elapsed_sec": elapsed,
            "iperf_sender_mbps": iptxt["sender_mbps"],
            "queue_measured_mbps": q_mbps,
            "queue_delta_bytes": q_bytes,
            "flow_delta_packets": f_pkts_delta,
            "flow_delta_bytes": f_bytes_delta,
            "flow_measured_mbps": f_mbps,
            "verdict": verdict
        }
    
    # --- FIX: 'finally' block is now correctly associated with the main 'try' block ---
    # 8. Cleanup Pinning Flows (runs whether the 'try' block succeeded or failed)
    finally:
        print(f"Cleaning up {len(pushed_pin_flows)} pinning flows...")
        unpin_path_floodlight(pushed_pin_flows)

# --- Floodlight-compatible Pinning Functions ---

def _pin_selector_floodlight(direction: str, protocol: str, src_ip: str, dst_ip: str, dst_port: int):
    """Builds a flat match dictionary for Floodlight."""
    # is_udp = protocol.lower() == "udp" # No longer needed
    
    # --- MODIFIED ---
    # We remove L4 (TCP/UDP) matching from the pinning flows.
    # This allows the TCP control channel AND the UDP data to use the same path.
    match = {
        "eth_type": "0x0800",
        # "ip_proto": "17" if is_udp else "6"  # <-- REMOVED
    }
    
    if direction == "forward":
        match["ipv4_src"] = f"{src_ip}/32"
        match["ipv4_dst"] = f"{dst_ip}/32"
        # if is_udp:                          # <-- REMOVED
        #     match["udp_dst"] = str(dst_port)
        # else:
        #     match["tcp_dst"] = str(dst_port)
    else: # reverse
        match["ipv4_src"] = f"{dst_ip}/32"
        match["ipv4_dst"] = f"{src_ip}/32"
        # if is_udp:                          # <-- REMOVED
        #     match["udp_src"] = str(dst_port)
        # else:
        #     match["tcp_src"] = str(dst_port)
    return match

def _pin_selector_floodlight_old(direction: str, protocol: str, src_ip: str, dst_ip: str, dst_port: int):
    """Builds a flat match dictionary for Floodlight."""
    is_udp = protocol.lower() == "udp"
    match = {
        "eth_type": "0x0800",
        "ip_proto": "17" if is_udp else "6"
    }
    
    if direction == "forward":
        match["ipv4_src"] = f"{src_ip}/32"
        match["ipv4_dst"] = f"{dst_ip}/32"
        if is_udp:
            match["udp_dst"] = str(dst_port)
        else:
            match["tcp_dst"] = str(dst_port)
    else: # reverse
        match["ipv4_src"] = f"{dst_ip}/32"
        match["ipv4_dst"] = f"{src_ip}/32"
        if is_udp:
            match["udp_src"] = str(dst_port)
        else:
            match["tcp_src"] = str(dst_port)
    return match

def install_pins_from_plan_floodlight(pins: list, src_ip: str, dst_ip: str, dst_port: int, protocol: str) -> list:
    """
    Install the pin flows (Floodlight format) and return the list of installed flows.
    """
    pushed_flows = []
    
    for i, p in enumerate(pins):
        # dev_dpid is already the colon-hex string
        dev_dpid = p["deviceId"] 
        outp = str(p["out_port"])
        direction = p["direction"]
        
        flow_name = f"pin-{direction}-{i}-{secrets.token_hex(4)}"
        
        flow_body = {
            "switch": dev_dpid,
            "name": flow_name,
            "priority": "65000",
            "active": "true",
            "actions": f"output={outp}"
        }
        
        # Add match criteria
        flow_body.update(_pin_selector_floodlight(direction, protocol, src_ip, dst_ip, dst_port))
        
        try:
            add_flow_floodlight(flow_body)
            pushed_flows.append(flow_body) # Store the whole flow for deletion
        except Exception as e:
            print(f"Warning: Failed to install pin flow: {e}")
            
    return pushed_flows

def unpin_path_floodlight(pushed_flows: list):
    """
    Remove all pinned rules by their 'name'.
    """
    for flow in pushed_flows:
        delete_flow_floodlight(flow["name"])

In [1025]:
# --- New helpers to satisfy the “missing src/dst” rules ---

def choose_dst_for_port(device_id: str, port_no: int) -> str:
    """
    Pick a destination IP that makes forward traffic EXIT on (device_id, port_no).
    Uses your diamond wiring.
    """
    # Final-hop host ports first
    if device_id == SW_OF["1"] and port_no == 3:  # s1 -> h1
        return HOSTS["h1"]
    if device_id == SW_OF["1"] and port_no == 4:  # s1 -> h2
        return HOSTS["h2"]
    if device_id == SW_OF["4"] and port_no == 3:  # s4 -> h3
        return HOSTS["h3"]
    if device_id == SW_OF["4"] and port_no == 4:  # s4 -> h4
        return HOSTS["h4"]

    # Inter-switch egress: choose a host "behind" the far side so packets must traverse this link
    # s1:1->s2 (right), s1:2->s3 (right) → pick a right-side host (h3 default)
    if device_id == SW_OF["1"] and port_no in (1, 2):
        return HOSTS["h3"]
    # s2:2->s4 (right) → pick right-side host; s2:1->s1 (left) → pick left-side host
    if device_id == SW_OF["2"] and port_no == 2:
        return HOSTS["h3"]
    if device_id == SW_OF["2"] and port_no == 1:
        return HOSTS["h1"]
    # s3:2->s4 (right) → right host; s3:1->s1 (left) → left host
    if device_id == SW_OF["3"] and port_no == 2:
        return HOSTS["h3"]
    if device_id == SW_OF["3"] and port_no == 1:
        return HOSTS["h1"]
    # s4 inter-switch (rare in your tests): port1->s2 (left) pick left host; port2->s3 (left) pick left host
    if device_id == SW_OF["4"] and port_no in (1, 2):
        return HOSTS["h1"]

    # Fallback
    return HOSTS["h3"]

def fill_missing_endpoints(plan: dict, device_id: str, port_no: int) -> tuple[str, str]:
    """
    Apply your policy:
      - If only dst is present → choose src that forces egress at (device,port)
      - If only src is present → choose dst that forces egress at (device,port)
      - If both missing → pick both to force egress at (device,port)
      - Ensure src != dst
    """
    src = plan.get("src_ip")
    dst = plan.get("dst_ip")

    # Normalize empty strings to None
    src = src or None
    dst = dst or None

    if dst is None:
        dst = choose_dst_for_port(device_id, port_no)
    if src is None:
        src = choose_src_for_port(device_id, port_no)

    # If they accidentally collide, flip src to the opposite side
    if src == dst:
        # If dst is on right side, move src to left; else move to right
        try:
            dst_edge, _ = _edge_for_ip(dst)
        except Exception:
            dst_edge = SW_OF["4"]  # assume right if unknown
        src = HOSTS["h1"] if dst_edge in (SW_OF["3"], SW_OF["4"]) else HOSTS["h3"]

    return src, dst



# --- QoS Topology-Aware Helpers ---
# (These helpers use the maps defined in Cell 3)

def _strip32(ip: str) -> str:
    return ip.split("/", 1)[0] if ip else None

def find_edge_for_ip(dst_ip: str):
    """Return (deviceId_str, portNo) for the host that owns dst_ip."""
    ip = _strip32(dst_ip)
    if ip in HOST_ATTACH:
        return HOST_ATTACH[ip]
    raise RuntimeError(f"Could not infer edge device/port for dst_ip={dst_ip}")

def infer_enforcement_point(plan):
    """
    Decide where to enforce if device/port not fully specified.
    """
    device_id, port_no = plan.get("device_id_str"), plan.get("port_no")
    if device_id and port_no:
        return device_id, port_no
    
    # Fallback: enforce at the final hop to the destination host
    return find_edge_for_ip(plan["dst_ip"])

def _is(dev: str, n: str) -> bool:
    """Checks if dev_dpid_str matches SW_OF[n]"""
    return dev == SW_OF.get(n)

def choose_dst_for_port(device_id_str: str, port_no: int) -> str:
    """Pick a destination IP that makes forward traffic EXIT on (device_id_str, port_no)."""
    if _is(device_id_str, "1") and port_no == 3: return HOSTS["h1"]
    if _is(device_id_str, "1") and port_no == 4: return HOSTS["h2"]
    if _is(device_id_str, "4") and port_no == 3: return HOSTS["h3"]
    if _is(device_id_str, "4") and port_no == 4: return HOSTS["h4"]

    if _is(device_id_str, "1") and port_no in (1, 2): return HOSTS["h3"]
    if _is(device_id_str, "2") and port_no == 2: return HOSTS["h3"]
    if _is(device_id_str, "2") and port_no == 1: return HOSTS["h1"]
    if _is(device_id_str, "3") and port_no == 2: return HOSTS["h3"]
    if _is(device_id_str, "3") and port_no == 1: return HOSTS["h1"]
    if _is(device_id_str, "4") and port_no in (1, 2): return HOSTS["h1"]
    return HOSTS["h3"] # Fallback

def choose_src_for_port(device_id_str: str, port_no: int) -> str:
    """Pick a source IP that will make *forward* traffic egress on (device_id_str, port_no)."""
    if _is(device_id_str, "1"):
        if port_no in (3, 4): return HOSTS["h3"]
        elif port_no in (1, 2): return HOSTS["h1"]
    if _is(device_id_str, "2"):
        return HOSTS["h1"] if port_no == 2 else HOSTS["h3"]
    if _is(device_id_str, "3"):
        return HOSTS["h1"] if port_no == 2 else HOSTS["h3"]
    if _is(device_id_str, "4"):
        if port_no in (3, 4): return HOSTS["h1"]
        elif port_no in (1, 2): return HOSTS["h3"]
    return HOSTS["h1"] # Fallback

def fill_missing_endpoints(plan: dict, device_id_str: str, port_no: int) -> tuple[str, str]:
    """Fill missing src/dst IPs based on topology."""
    src = plan.get("src_ip") or None
    dst = plan.get("dst_ip") or None

    if dst is None:
        dst = choose_dst_for_port(device_id_str, port_no)
    if src is None:
        src = choose_src_for_port(device_id_str, port_no)

    if src == dst:
        # Flip src to the opposite side
        dst_edge, _ = find_edge_for_ip(dst)
        src = HOSTS["h1"] if dst_edge in (SW_OF["3"], SW_OF["4"]) else HOSTS["h3"]
    return src, dst

def make_pin_path_flows(device_id_str: str, port_no: int,
                        src_ip: str, dst_ip: str, dst_port: int,
                        protocol: str = "tcp"):
    """
    Build a pinning plan (list of dicts) so src_ip -> dst_ip forward traffic
    *must* traverse (device_id_str, port_no).
    
    This version correctly installs the reverse path for BOTH TCP and UDP
    to allow iperf server to reply to the client.
    """
    proto = protocol.lower()
    is_udp = (proto == "udp")
    pins = []
    src_edge_dev, src_edge_port = find_edge_for_ip(src_ip)
    dst_edge_dev, dst_edge_port = find_edge_for_ip(dst_ip)

    def add(dev_key, outp, direction):
        pins.append({"deviceId": SW_OF[dev_key], "out_port": int(outp), "direction": direction})

    # --- cases by enforcement point ---
    
    # s4 -> h3/h4
    if _is(device_id_str, "4") and port_no in (3, 4): 
        if _is(src_edge_dev, "1"):
            add("1", 1, "forward") # s1 -> s2
            add("2", 2, "forward") # s2 -> s4
            # Reverse path
            add("4", 1, "reverse") # s4 -> s2
            add("2", 1, "reverse") # s2 -> s1
            add("1", src_edge_port, "reverse") # s1 -> h1/h2
    
    # s1 -> h1/h2        
    elif _is(device_id_str, "1") and port_no in (3, 4): 
        if _is(src_edge_dev, "4"):
            add("4", 2, "forward") # s4 -> s3
            add("3", 1, "forward") # s3 -> s1
            # Reverse path
            add("1", 2, "reverse") # s1 -> s3
            add("3", 2, "reverse") # s3 -> s4
            add("4", src_edge_port, "reverse") # s4 -> h3/h4

    # s1 -> s3
    elif _is(device_id_str, "1") and port_no == 2: 
        add("1", 2, "forward")
        add("3", 2, "forward")
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        # Reverse path
        add("4", 2, "reverse")
        add("3", 1, "reverse")
        add("1", src_edge_port, "reverse")
    
    # s1 -> s2        
    elif _is(device_id_str, "1") and port_no == 1: 
        add("1", 1, "forward")
        add("2", 2, "forward")
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        # Reverse path
        add("4", 1, "reverse")
        add("2", 1, "reverse")
        add("1", src_edge_port, "reverse")
    
    # s2 -> s4
    elif _is(device_id_str, "2") and port_no == 2: 
        add("1", 1, "forward")   # s1 -> s2
        add("2", 2, "forward")   # s2 -> s4
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        # Reverse path
        add("4", 1, "reverse")
        add("2", 1, "reverse")
        add("1", src_edge_port, "reverse")
            
    # s3 -> s4
    elif _is(device_id_str, "3") and port_no == 2: 
        add("1", 2, "forward")   # s1 -> s3
        add("3", 2, "forward")   # s3 -> s4
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        # Reverse path
        add("4", 2, "reverse")
        add("3", 1, "reverse")
        add("1", src_edge_port, "reverse")
            
    else: # Fallback (e.g., h1->s1->s2->s4->h3)
        add("1", 1, "forward") # s1->s2
        add("2", 2, "forward") # s2->s4
        add("4", dst_edge_port, "forward") # s4 -> h3
        # Reverse path
        add("4", 1, "reverse")
        add("2", 1, "reverse")
        add("1", src_edge_port, "reverse")

    # --- Filter pins ---
    final_pins = []
    seen = set()
    for p in pins:
        # Don't overshadow the QoS rule on the enforcement port
        if (p["direction"] == "forward" and 
            p["deviceId"] == device_id_str and 
            int(p["out_port"]) == int(port_no)):
            continue
            
        key = (p["deviceId"], int(p["out_port"]), p["direction"])
        if key not in seen:
            seen.add(key)
            final_pins.append(p)
            
    return final_pins

def make_pin_path_flows_old2(device_id_str: str, port_no: int,
                        src_ip: str, dst_ip: str, dst_port: int,
                        protocol: str = "tcp"):
    """
    Build a pinning plan (list of dicts) so src_ip -> dst_ip forward traffic
    *must* traverse (device_id_str, port_no).
    """
    proto = protocol.lower()
    is_udp = (proto == "udp")
    pins = []
    src_edge_dev, src_edge_port = find_edge_for_ip(src_ip)
    dst_edge_dev, dst_edge_port = find_edge_for_ip(dst_ip)

    def add(dev_key, outp, direction):
        pins.append({"deviceId": SW_OF[dev_key], "out_port": int(outp), "direction": direction})

    # --- cases by enforcement point ---
    if _is(device_id_str, "4") and port_no in (3, 4): # s4 -> h3/h4
        if _is(src_edge_dev, "1"):
            add("1", 1, "forward") # s1 -> s2
            add("2", 2, "forward") # s2 -> s4
        if not is_udp and _is(src_edge_dev, "1"):
            add("4", 1, "reverse") # s4 -> s2
            add("2", 1, "reverse") # s2 -> s1
            add("1", src_edge_port, "reverse") # s1 -> h1/h2
            
    elif _is(device_id_str, "1") and port_no in (3, 4): # s1 -> h1/h2
        if _is(src_edge_dev, "4"):
            add("4", 2, "forward") # s4 -> s3
            add("3", 1, "forward") # s3 -> s1
        if not is_udp and _is(src_edge_dev, "4"):
            add("1", 2, "reverse") # s1 -> s3
            add("3", 2, "reverse") # s3 -> s4
            add("4", src_edge_port, "reverse") # s4 -> h3/h4

    # --- START OF FIX ---
    # The inter-switch paths must include the final hop to the destination host
    
    elif _is(device_id_str, "1") and port_no == 2: # s1 -> s3
        add("1", 2, "forward") # s1 -> s3 (will be filtered)
        add("3", 2, "forward") # s3 -> s4
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        if not is_udp:
            add("4", 2, "reverse")
            add("3", 1, "reverse")
            add("1", src_edge_port, "reverse")
            
    elif _is(device_id_str, "1") and port_no == 1: # s1 -> s2
        add("1", 1, "forward") # s1 -> s2 (will be filtered)
        add("2", 2, "forward") # s2 -> s4
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        if not is_udp:
            add("4", 1, "reverse")
            add("2", 1, "reverse")
            add("1", src_edge_port, "reverse")
    
    elif _is(device_id_str, "2") and port_no == 2: # s2 -> s4
        add("1", 1, "forward")   # s1 -> s2
        add("2", 2, "forward")   # s2 -> s4 (will be filtered)
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        if not is_udp:
            add("4", 1, "reverse")
            add("2", 1, "reverse")
            add("1", src_edge_port, "reverse")
            
    elif _is(device_id_str, "3") and port_no == 2: # s3 -> s4
        add("1", 2, "forward")   # s1 -> s3
        add("3", 2, "forward")   # s3 -> s4 (will be filtered)
        add("4", dst_edge_port, "forward") # s4 -> h3/h4
        if not is_udp:
            add("4", 2, "reverse")
            add("3", 1, "reverse")
            add("1", src_edge_port, "reverse")
            
    # (Other inter-switch paths like s2->s1 or s3->s1 can be added if needed)
    # --- END OF FIX ---

    else: # Fallback (e.g., h1->s1->s2->s4->h3)
        add("1", 1, "forward") # s1->s2
        add("2", 2, "forward") # s2->s4
        add("4", dst_edge_port, "forward") # s4 -> h3
        if not is_udp:
            add("4", 1, "reverse")
            add("2", 1, "reverse")
            add("1", src_edge_port, "reverse")

    # --- Filter pins ---
    final_pins = []
    seen = set()
    for p in pins:
        # Don't overshadow the QoS rule on the enforcement port
        if (p["direction"] == "forward" and 
            p["deviceId"] == device_id_str and 
            int(p["out_port"]) == int(port_no)):
            continue
        
        # For UDP, only keep forward pins
        if is_udp and p["direction"] != "forward":
            continue
            
        key = (p["deviceId"], int(p["out_port"]), p["direction"])
        if key not in seen:
            seen.add(key)
            final_pins.append(p)
            
    return final_pins

def make_pin_path_flows_old(device_id_str: str, port_no: int,
                        src_ip: str, dst_ip: str, dst_port: int,
                        protocol: str = "tcp"):
    """
    Build a pinning plan (list of dicts) so src_ip -> dst_ip forward traffic
    *must* traverse (device_id_str, port_no).
    """
    proto = protocol.lower()
    is_udp = (proto == "udp")
    pins = []
    src_edge_dev, src_edge_port = find_edge_for_ip(src_ip)
    dst_edge_dev, dst_edge_port = find_edge_for_ip(dst_ip)

    def add(dev_key, outp, direction):
        pins.append({"deviceId": SW_OF[dev_key], "out_port": int(outp), "direction": direction})

    # --- cases by enforcement point ---
    if _is(device_id_str, "4") and port_no in (3, 4): # s4 -> h3/h4
        if _is(src_edge_dev, "1"):
            add("1", 1, "forward") # s1 -> s2
            add("2", 2, "forward") # s2 -> s4
        if not is_udp and _is(src_edge_dev, "1"):
            add("4", 1, "reverse") # s4 -> s2
            add("2", 1, "reverse") # s2 -> s1
            add("1", src_edge_port, "reverse") # s1 -> h1/h2
            
    elif _is(device_id_str, "1") and port_no in (3, 4): # s1 -> h1/h2
        if _is(src_edge_dev, "4"):
            add("4", 2, "forward") # s4 -> s3
            add("3", 1, "forward") # s3 -> s1
        if not is_udp and _is(src_edge_dev, "4"):
            add("1", 2, "reverse") # s1 -> s3
            add("3", 2, "reverse") # s3 -> s4
            add("4", src_edge_port, "reverse") # s4 -> h3/h4

    elif _is(device_id_str, "1") and port_no == 2: # s1 -> s3
        add("1", 2, "forward")
        add("3", 2, "forward")
        if not is_udp:
            add("4", 2, "reverse")
            add("3", 1, "reverse")
            add("1", src_edge_port, "reverse")
            
    elif _is(device_id_str, "1") and port_no == 1: # s1 -> s2
        add("1", 1, "forward")
        add("2", 2, "forward")
        if not is_udp:
            add("4", 1, "reverse")
            add("2", 1, "reverse")
            add("1", src_edge_port, "reverse")
    
    # (Add other inter-switch cases if needed, e.g., s2, s3)

    else: # Fallback
        add("1", 1, "forward") # s1->s2
        add("2", 2, "forward") # s2->s4
        if not is_udp:
            add("4", 1, "reverse")
            add("2", 1, "reverse")
            add("1", src_edge_port, "reverse")

    # --- Filter pins ---
    final_pins = []
    seen = set()
    for p in pins:
        # Don't overshadow the QoS rule on the enforcement port
        if (p["direction"] == "forward" and 
            p["deviceId"] == device_id_str and 
            int(p["out_port"]) == int(port_no)):
            continue
        
        # For UDP, only keep forward pins
        if is_udp and p["direction"] != "forward":
            continue
            
        key = (p["deviceId"], int(p["out_port"]), p["direction"])
        if key not in seen:
            seen.add(key)
            final_pins.append(p)
            
    return final_pins

def extract_port_number(text: str):
    """
    Extract the Ethernet port number from a natural language text.
    
    Parameters:
        text (str): The input text containing the port reference.
    
    Returns:
        int: Extracted port number or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match explicit numbers after keywords
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\d+)', text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Match ordinal words (e.g., 'second port', 'third interface')
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\w+)', text, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return ordinals[ordinal_word]

    # Match standalone ordinal words (e.g., 'second')
    for word, number in ordinals.items():
        if word in text.lower():
            return number

    return None

def parse_intent_text(slicing_info, src_ip, dst_ip):
    """
    Parses the LLM slicing output into a structured plan.
    """
    slicing_queue_id = slicing_info['queue_id'] if (slicing_info['queue_id']) != "" else 1
    proto = slicing_info['traffic_type'] if "udp" in slicing_info['traffic_type'] else ("tcp" if "tcp" in slicing_info['traffic_type'] or "http" in slicing_info['traffic_type'] else "tcp")
    slicing_l4_port = slicing_info['l4_port'] if slicing_info['l4_port'] != "" else 80
    port_no = extract_port_number(slicing_info['port_id'])
    
    # Use the Floodlight dpid extractor
    device_id_str = extract_dpid_string_floodlight(slicing_info['switch_id'])

    #print("\n***************\n", proto, "\n*************************\n")

    return {
        "protocol": proto,
        "dst_port": int(slicing_l4_port),
        "device_id_str": device_id_str, # colon-hex string (or None)
        "port_no": port_no,             # integer (or None)
        "queue_id": int(slicing_queue_id),
        "src_ip": src_ip,               # may be None
        "dst_ip": dst_ip,               # may be None
    }

def intent_to_verifier_args(slicing_info, old_src_ip, old_dst_ip, 
                            main_flow_name: str, 
                            main_static_flow_rule: dict,
                            target_mbps: float = 4.0):
    """
    Parses slicing info and the main flow rule to create
    arguments for the verify_qos_flow_with_iperf function.
    """
    
    # 1. Parse the text intent
    plan = parse_intent_text(slicing_info, old_src_ip, old_dst_ip)
    
    # 2. Check for in_port to lock the source host
    if main_static_flow_rule:
        in_port = main_static_flow_rule.get("in_port")
        if in_port:
            # Build reverse map: {(dpid_str, port_num): ip}
            port_to_ip = {v: k for k, v in HOST_ATTACH.items()}
            
            # The device_id_str is from the *intent*. The rule's
            # "switch" field is the source of truth.
            rule_device_str = main_static_flow_rule.get("switch")
            
            host_ip = port_to_ip.get( (rule_device_str, int(in_port)) )
            
            if host_ip:
                print(f"[INFO] Rule has in_port:{in_port}, locking source host to {host_ip}")
                plan["src_ip"] = host_ip
            else:
                print(f"[WARN] Rule has in_port:{in_port}, but no host is mapped to ({rule_device_str}, {in_port}).")

    # 3. Find the enforcement point (device/port for the queue)
    # If dst_ip is now set (from 'plan'), use it
    if plan["dst_ip"] and not (plan["device_id_str"] and plan["port_no"]):
         device_id_str, port_no = find_edge_for_ip(plan["dst_ip"])
    else:
         device_id_str, port_no = infer_enforcement_point(plan)
        
    plan["device_id_str"], plan["port_no"] = device_id_str, port_no

    # 4. Fill any remaining missing endpoints
    src_ip, dst_ip = fill_missing_endpoints(plan, device_id_str, port_no)
    plan["src_ip"], plan["dst_ip"] = src_ip, dst_ip

    # 5. Get integer DPID for OVS/Mininet helpers
    dpid_int = int(device_id_str.split(":")[-1], 16) 

    # 6. Build verifier args
    args = {
        "flow_device_dpid_str": device_id_str,
        "flow_name":            main_flow_name,
        "queue_device_dpid_int":dpid_int,
        "queue_port_no":        port_no,
        "queue_id":             plan["queue_id"],
        "src_ip":               src_ip,
        "dst_ip":               dst_ip,
        "dst_port":             plan["dst_port"],
        "target_mbps":          target_mbps,
        "protocol":             plan["protocol"],
        "main_static_flow_rule": main_static_flow_rule
    }

    # 7. Build Pin plan
    pins_plan = make_pin_path_flows(
        device_id_str, 
        port_no, 
        src_ip, 
        dst_ip, 
        plan["dst_port"],
        plan["protocol"]
    )
    
    return args, pins_plan, plan

In [ ]:
# done intent = "Forward TCP traffic on port 80 destined for 10.0.1.3 via interface 3, assigning it to queue 1 for prioritized handling in switch 4."
# done intent = "In switch 4, traffic destined for 10.0.1.4 should use port 4."
# done intent =  "In switch 4, block all IPv4 traffic from 10.0.1.1 to 10.0.1.4 with a high priority, ensuring the switch operates as a firewall."
# done intent = "Drop all traffic from 10.0.1.3 on switch 2 while forwarding all other traffic normally."
# done intent = "If interface 1 on node 2 receives UDP to port 80, pass via port 2, queue 1."
# done intent = "If interface 1 on node 2 receives UDP to port 5201, pass via port 2, queue 1."
# done intent = "In node 1, traffic destined for 10.0.1.2 should use port 4."
# done intent = "In switch 2, traffic from port 1 should pass through port 2."
# done intent = "Traffic from port 2 of switch 2 to 10.0.1.1 should use interface 1."
# done intent = "In switch 4, traffic from 10.0.1.1 to 10.0.1.4 should use output interface 4."

In [ ]:
current_time = time.time()

slicing_info = run_LLM_Slice(intent)

# This function is OVS-based and fine
# create_two_queue_for_switch_handler(slicing_info)

# device_id here is the integer DPID
deployment_status, flow_id, device_id_int, translated_flow_rule, operational_flow_rule = end_to_end_IBN(intent)

if 'use_queue' in slicing_info and slicing_info['use_queue'] == 1 and deployment_status:
    #Creating one queue
    expected_queue_rate_mbps = 4.0
    port_max = 100_000_000
    min_rate = max_rate = int(expected_queue_rate_mbps * 1_000_000)
    slicing_queue_id = slicing_info['queue_id'] if (slicing_info['queue_id']) != "" else 1
    port_no = extract_port_number(slicing_info['port_id'])
    
    # We need the integer DPID for this OVS helper
    ensure_qos_cap(device_id_int, port_no, slicing_queue_id, min_bps=min_rate, max_bps=max_rate, port_cap_bps=port_max)

global llm_caller_flag

if (deployment_status == True):
        proc_time_s = (time.time() - current_time)
        print(f"\n\nSuccessfully translated and installed the rule in Floodlight Controller. Time taken: {proc_time_s:.2f}s")
        print("\nThe translated flow rule is: ", translated_flow_rule)
        
        # 'flow_id' is the flow 'name'
        # 'device_id_int' is the integer DPID
        
        src_host, dst_host, src_ip, dst_ip = extract_host_and_ip_floodlight(translated_flow_rule)
        flow_rule_type, flow_rule_specificity = classify_floodlight_flow_rule(translated_flow_rule)
        
        append_intent_to_store(
            "IntentStore_Floodlight.jsonl", # <-- Use a new store file
            nl_intent=intent,
            json_flow_rule=translated_flow_rule,
            device_id=translated_flow_rule.get("switch"), # Store DPID string
            flow_id=flow_id, # Store flow name
            intent_type=flow_rule_type,
            intent_specificity=flow_rule_specificity
            )
        
        if (flow_rule_type== "security"):
            # Get the DPID string for the API
            dpid_str = translated_flow_rule.get("switch")
            ping_count, candidate_src_ip, candidate_dst_ip, ping_output = floodlight_assurance_for_security_intent(src_ip, dst_ip, device_id_int)
            
            if (llm_caller_flag == 1):
                print("\nAsking LLM to generate corrective actions...")
                assurance_LLM_prompt = generate_corrective_action_prompt_floodlight(intent, operational_flow_rule, dpid_str, ping_count,
                                    candidate_src_ip, candidate_dst_ip, ping_output)
                llm_response = Run_assurance_LLM (assurance_LLM_prompt)
                print(llm_response)
                parse_and_execute_corrective_actions_floodlight(operational_flow_rule, llm_response, device_id_int)


        elif (flow_rule_type== "qos"):
            
            target_mbps = 4.0
            
            # flow_id is the 'name'
            main_flow_name = flow_id 

            # NEW LINE:
            # We pass 'operational_flow_rule' which has the correct "0x11"
            #args, pins_plan, plan = intent_to_verifier_args(slicing_info, src_ip, dst_ip, main_flow_name, operational_flow_rule, target_mbps)
            
            
            args, pins_plan, plan = intent_to_verifier_args(
                slicing_info, 
                src_ip, 
                dst_ip, 
                main_flow_name, 
                translated_flow_rule, # Pass the rule itself
                target_mbps
            )
            protocol = plan["protocol"]
            dst_port = args["dst_port"]

            try:
                # 2) Call the verifier
                verify_qos_flow_with_iperf(
                    flow_device_dpid_str=args["flow_device_dpid_str"],
                    flow_name=args["flow_name"], 
                    queue_device_dpid_int=args["queue_device_dpid_int"],
                    queue_port_no=args["queue_port_no"],
                    queue_id=args["queue_id"],
                    src_ip=args["src_ip"],
                    dst_ip=args["dst_ip"],
                    dst_port=dst_port,
                    target_mbps=args["target_mbps"],
                    protocol=protocol,
                    pin_path_flows=pins_plan,
                    main_static_flow_rule=args["main_static_flow_rule"],
                    
                    # --- Explicit iperf parameters ---
                    duration_sec=8,
                    parallel=8 if protocol == "tcp" else 1,
                    tcp_mss=1200,
                    tolerance_pct=10.0,
                    udp_bw_mbps=50.0,
                    udp_len_bytes=1200
                )
            except Exception as e:
                print(f"\n--- QoS VERIFICATION FAILED ---")
                print(f"An error occurred during QoS verification: {e}")
                import traceback
                traceback.print_exc()
            finally:
                # Verifier has its own cleanup, but we pass
                pass    
             
        elif (flow_rule_type== "forwarding"):
             # You can implement a forwarding assurance test here (e.g., ping)
             print(f"\n[INFO] Forwarding rule {flow_id} installed.")
             # ONOS_assurance_for_forwarding_intent(src_host, dst_host, src_ip, dst_ip)

        elapsed_time = (time.time() - current_time)
        print(f"\nTime taken for end-to-end IBN: {elapsed_time:.2f}s")

elif (flow_id == "Tie"):
    print("\n\nReport to the operator about this conflict resolution issue. Need adjustment to conflict resolution policy.\n")

elif (flow_id == "existing_rule_win"):
    print("\n\nThe new intent conflicts with an existing one having a higher priority according to current policy, hence the new intent was not installed. See the existing flow rule that conflicts.\n")

else:
    print("\n\nLLM failed to produce meaningful response. Either update context example or model.\n")

In [1028]:
# flow_id = "intent_45-38f99444"

In [ ]:
print(f"\n--- Attempting to delete flow: {flow_id} ---")
try:
    # Call the delete function from [Cell 4]
    delete_success = delete_flow_floodlight(flow_id)
    
    if delete_success:
        print("--- Deletion Successful ---")
    else:
        print("--- Deletion Failed (see warning above) ---")
        
except Exception as e:
    print(f"An error occurred during deletion: {e}")